# Data Fusion Contest

## Решение задачи `Goodsification`

* February 2021 - March 2021
* Александр Широков, [github](https://github.com/aptmess)

In [14]:
import re
import pickle
import string
import warnings
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import warnings
from download_text import get_grams_from_text
from corus.sources.meta import METAS
from corus.readme import format_metas, show_html, patch_readme
from corus import load_lenta, load_wiki
warnings.filterwarnings("ignore")

%matplotlib inline
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aptmess/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aptmess/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/aptmess/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/aptmess/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Загрузим данные.

In [15]:
def load_data(path_to_file):
    train = pd.read_parquet(path_to_file)
    map_column_to_type = {
        'category_id': np.int16,
        'item_nds_rate': np.int16,
        'item_price': np.int16,
        'item_quantity': np.float32,
        'receipt_dayofweek': np.int16,
        'receipt_id': np.int32}
    for key in map_column_to_type:
        train[key] = train[key].astype(map_column_to_type[key])
    train = train.drop('brands', axis=1)
    return train

In [16]:
%%time
train_data_name = '../../conference_data/data_fusion_train.parquet'
train = load_data(train_data_name)

CPU times: user 1min 40s, sys: 45.4 s, total: 2min 25s
Wall time: 1min 41s


Рассмотрим размеченные категории товаров.

In [17]:
train_data = train[train['category_id'] != -1]

### Формирование словаря

```bash
!wget -P ~/IHaskell/word_data/ wget https://dumps.wikimedia.org/ruwiki/latest/ruwiki-latest-pages-articles.xml.bz2
```

```python
g = get_grams_from_text(path='../../word_data/ruwiki-latest-pages-articles.xml.bz2',
                        n=[1], 
                        amount_of_sentense=20000, 
                        show_how_much=100, 
                        delete_stop_words=False)
pickle.dump(g, open("../../word_data/words_wiki_1.pkl", "wb"))
```

In [ ]:
word_wiki = get_grams_from_text(path='../../word_data/ruwiki-latest-pages-articles.xml.bz2',
                                n=[1], 
                                amount_of_sentense=200000, 
                                show_how_much=5000, 
                                delete_stop_words=False)

In [ ]:
pickle.dump(word_wiki, open("../../word_data/words_wiki_one_million.pkl", "wb"))

**Словарь из `train_data`**:

```python
unique_values_train__item_name = train['item_name'].unique()
word = Counter()
for num, i in enumerate(unique_values_train__item_name):
    if num % 100000 == 0:
        print(num)
    word.update(tokens(i, language='rus'))
pickle.dump(g, open("../../word_data/words_train.pkl", "wb")) 
```

### Загрузка словаря

```python
g = pickle.load(open("../../word_data/words_wiki_1.pkl", "rb"))
w = pickle.load(open("../../word_data/words.pkl", "rb"))
z = pickle.load(open("../../word_data/words_train.pkl", "rb"))
```

In [18]:
g = pickle.load(open("../../word_data/words_wiki_1.pkl", "rb"))
w = pickle.load(open("../../word_data/words.pkl", "rb"))
z = pickle.load(open("../../word_data/words_train.pkl", "rb"))

In [19]:
COUNTS = w[1] + g[1]

## Исправление опечаток / пробелов

In [20]:
from collections import Counter

def pdist(counter):
    "Превращает частоты из Counter в вероятностное распределение."
    N = sum(list(counter.values()))
    return lambda x: counter[x] / N

P = pdist(COUNTS)
ALPHABET = {'ENG': 'abcdefghijklmnopqrstuvwxyz',
            'RUS': 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'}

def memo(f):
        "Запомнить результаты исполнения функции f, чьи аргументы args должны быть хешируемыми."
        cache = {}
        def fmemo(*args):
            if args not in cache:
                cache[args] = f(*args)
            return cache[args]
        fmemo.cache = cache
        return fmemo


def tokens(text, language):
        shab = r'[a-z]+' if language == 'eng' else r'[а-я]+'
        """Возвращает список токенов (подряд идущих буквенных последовательностей) в тексте. 
           Текст при этом приводится к нижнему регистру."""
        return re.findall(shab, text.lower())





def Pwords(words):
    "Вероятности слов, при условии, что они независимы."
    return product(P(w) for w in words)


def product(nums):
    "Перемножим числа.  (Это как `sum`, только с умножением.)"
    result = 1
    for x in nums:
        result *= x
    return result


def splits(text, start=0, L=29):
    "Вернуть список всех пар (a, b); start <= len(a) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]


@memo
def segment(text):
    "Вернуть список слов, который является наиболее вероятной сегментацией нашего текста."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)

In [21]:
un_values = train_data['item_name'].unique()

In [22]:
segme = {}
for i in un_values:
    spl = tokens(i, language='rus')
    segme[i] = ' '.join([' '.join(segment(tok)) for tok in spl])

In [23]:
train_data['segme'] = train_data['item_name'].map(segme)

## Исправление опечаток

In [24]:
def correct(word):
    """
    Поиск лучшего исправления ошибки для данного слова.
    
    :param word: слово для исправления
    :return: предрассчитать edit_distance == 0, затем 1, затем 2; в противном случае оставить слово "как есть"
    """
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

In [25]:
def known(words):
    "Вернуть подмножество слов, которое есть в нашем словаре."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    "Вернуть все строки, которые находятся на edit_distance == 0 от word (т.е., просто само слово)."
    return {word}

def edits2(word):
    "Вернуть все строки, которые находятся на edit_distance == 2 от word."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [26]:
def edits1(word):
    "Возвращает список всех строк на расстоянии edit_distance == 1 от word."
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    "Возвращает список всех возможных разбиений слова на пару (a, b)."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = ALPHABET['RUS']

In [27]:
def correct_text(text, language='rus'):
    text = text.lower()
    "Исправить все слова с опечатками в тексте."
    shab = '[a-zA-Z]+' if language == 'eng' else '[а-яА-Я]+'
    return re.sub(shab, correct_match, text)

def correct_match(match):
    "Исправить слово word в match-группе, сохранив регистр: upper/lower/title."
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    "Возвращает функцию регистра по тексту: upper, lower, title, или str."
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [29]:
un_values_segme = train_data['segme'].unique()

In [17]:
opechat = {}
for i in un_values_segme:
    opechat[i] = correct_text(i)

In [18]:
train_data['mistakes_checks'] = train_data['item_name'].map(segme)

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
x = train_data.drop_duplicates('mistakes_checks')

tfidf = TfidfVectorizer(
    max_features=800000, 
    ngram_range=(1, 6), 
    stop_words ='russian',
    analyzer="char_wb", 
    norm = 'l2'
)

x['new'] = x['mistakes_checks'] + ' ' + x['item_name'] + ' ' + x['segme']
X_train = tfidf.fit_transform(x.new)
y_train = x['category_id']
clf = LinearSVC()
folds = KFold(8, 
              shuffle=True, 
              random_state=0)

In [27]:
predicts = cross_val_predict(clf, 
                             X_train, 
                             y_train, 
                             cv=folds, 
                             n_jobs=8, 
                             method='predict')

In [28]:
from sklearn.metrics import f1_score

score = f1_score(y_train, predicts, average='weighted')
print(score)

0.8281417267550237


В своей работе я сделал ставку на предобработку текста и выуживание из него тех незаметных вещей, которые простые `regex` или токенизация не смогут найти.

In [31]:
opechat2 = {}
for i in un_values:
    opechat2[i] = corrector.FixFragment(i)
    print(i, '       ', opechat2[i])

Молоко 3,2%,шт         Молоко 3,2%,шт
Компот из изюма, 114 ккал         Компот из изюма, 114 ккал
Макаронные изделия отварные (масло сливочное), 240 ккал         Макаронные изделия товарные (масло сливочное), 240 ккал
Кофе Капучино Большой Эден 18         Кофе Капучино Большой Эден 18
Хлеб на СЫВОРОТКЕ 350г         Хлеб на СЫВОРОТКЕ 350г
Сосиска в тесте с сыром 1шт ГЕ         Сосиска в тесте с сыром 1шт ГЕ
ЛанчБаскет 5 за 300: 2 шт ОРИГ Стрипсы кур фил         ЛанчБаскет 5 за 300: 2 шт ОРИГ Стрипсы кур фил
Станд Картофель фри         Стан Картофель фри
Хот-дог Куриный СБ         Хот-дог Куриный СБ
Чизбургер с луком СБ         Чизбургер с луком СБ
Бумага туалетная Набер. Челны 1/48         Бумага туалетная Набор. Челны 1/48
Молоко пастерилиз. т/рекс 2,5 %  1 л. 1/8 БМК         Молоко пастерилиз. т/рекс 2,5 %  1 л. 1/8 БМК
Пакет         Пакет
Спинки варено-копченые из мяса ЦБ в/у охладенные         Спинки варено-копченые из мяса ЦБ в/у охлаждение
Курник 1 шт.         Курник 1 шт.
ТРК 02 

ЗЕЛПМ-КА32Х62Х17         ЗАТЕМ-НА32Х62Х17
Ср-во чист унив Селена Sanitol 500мл         Ср-во чист унив Селена Sanitol 500мл
Карамель леденц."ВеселыеДольки"микс400г         Карамель леденц."ВеселыеДольки"микс400г
Кетчуп томатный         Кетчуп томатный
Купон 5050: 4штКур. крылышки (Баскет Дуэт)         Купон 5050: 4шкУр. крылышки (Банкет Дуэт)
 К.Ц.Яйца куриные стол.С2 10шт          К.Ц.Яйца куриные стол.С2 10шт
 Хлеб ПШЕНИЧНЫЙ нар.325г          Хлеб ПШЕНИЧНЫЙ нар.325г
Капуста         Капуста
Кофе растворимый Маккофе Original 3в1 20г         Кофе растворимый Марокко Original 3в1 20г
Мороженое Дакгомз ванильное в глазури 8% рожок 60г         Мороженое Диагноз ванильное в глазури 8% рожок 60г
ЯЙЦО ЭКСТРА С0         ЯЙЦО ЭКСТРА С0
ПЕЧ/130Г/ЮБИ         ПЕЧ/130Г/БИ
ЯЙЦО КУРИНОЕ С1 ВАРА         ЯЙЦО КУРИНОЕ С1 ВАРА
Йогурт фруктовый "Вишня" 2,5%,шт         Йогурт фруктовый "Вишня" 2,5%,шт
АССОРТИМЕНТЕ, 300МЛ         АССОРТИМЕНТЕ, 300МЛ
Жевательная резинка Дирол арбузно-дынный коктейль 14 г    

ФРУКТЫ СВ. БАНАНЫ ЭКВАДОР 1КГ         ФРУКТЫ СВ. БАНАНЫ ЭКВАДОР 1КГ
Компот из ананасов (консервированных) 200г, 105 ккал         Компот из ананасов (консервированных) 200г, 105 ккал
Сметана 100г, 159 ккал         Сметана 100г, 159 ккал
Шаньга с картофелем 100г, 218 ккал (завод)         Шаньга с картофелем 100г, 218 ккал (завод)
Кабачки вес, кг         Кабачки вес, кг
Капуста Белокочанная 1кг         Капуста Белокочанная 1кг
Конфеты ЛЕВУШКА С КАРАМЕЛЬЮ ве         Конфеты ЛЕВУШКА С КАРАМЕЛЬЮ ве
МАТНАКАШ 400гр         МАТНАКАШ 400гр
Масло сливочн ДЕРЕВНЯ СЧАСТЛИВ         Масло сливочн ДЕРЕВНЯ СЧАСТЛИВ
ПОМИДОРЫ НА ВЕТКЕ 1кг         ПОМИДОРЫ НА ВЕТКЕ 1кг
Хлеб ОКСКИЙ 600гр Узловский ХК         Хлеб ОКСКИЙ 600гр Узловский ХК
Шпроты в масле крупные 160г ключ ж/б Беринг         Шпроты в масле крупные 160г ключ ж/б Беринг
3 шт.Сырные подушечки         3 шт.Сырые подушечки
ДВОЙНОЙ ЧИЗБУРГЕР         ДВОЙНОЙ ЧИЗБУРГЕР
КАРТОФЕЛЬ-ФРИ СТД.         КАРТОФЕЛЬ-ФРИ СТД.
ЧАЙ ЧЕР. ЭРЛ ГРЕЙ БОЛ         ЧАЙ Ч

КД/НЕКТАР АПЕЛЬС.2Л         КД/НЕКТАР АПРЕЛЯ.2Л
СТ ПОР АВТ СВЕЖ ЛАВА         СТ ПОР АВТ СВЕЖ ЛАВА
БатонЩЕЛКОВСК38гнар          БатонЩЕЛКОВСК38гнар 
Салат из моркови с яблоками         Салат из моркови с яблоками
ЙогуртНежныйПерсикБЗМЖ5% 110г ван         ЙогуртНежныйПерсикБЗМЖ5% 110г ван
Отбеливатель Белизна 1000мл         Отбеливатель Белизна 1000мл
800Г ХАЛВА РОТ ФРОНТ         800Г ХАЛВА РОТ ФРОНТ
Батончик Раз Два 30г Славянка         Батончик Раз Два 30г Славянка
Арбуз         Арбуз
ЧИПСЫ МАЛОСОЛ.ОГУРЧ         ЧИПСЫ МАЛОГО.МОГУТ
 Батон НАРЕЗНОЙ нарезка п/э 380г          Батон НАРЕЗНОЙ нарезка п/в 380г
Зелёный горошек С&B 310гр. СТМ         Зелёный горошек С&B 310гр. СТМ
Конфеты Марсианка тирамису 200гр.         Конфеты Марианна тиражами 200гр.
Конфеты"Ореховое царство" 300гр.Невс         Конфеты"Ореховое царство" 300гр.Нес
Чай черн. Принцесса ГИТА 1,2гХ100пак         Чай черн. Принцесса ГИТА 1,2иХ100пак
ГРИБЫ ВЕШЕНКА ФАС350         ГРИБЫ ВЕШЕНКА ФАС350
A18ADESOM03-AB 43-46 Носки для 

         ВЛ.САЛФ.АНТИБАК.20ШТ                  ВЛ.САЛА.АНТИФА.20ШТ
 К"Мясная"ПАПА МОЖЕТ!"500г          К"Мясная"ПАПА МОЖЕТ!"500г
 ХЛЕБ I/II СОРТ 600Г ИРКУТСКИЙ ХЛЕБОЗАВОД          ХЛЕБ I/II СОРТ 600Г ИРКУТСКИЙ ХЛЕБОЗАВОД
Лепешка Узбекская 330г Тандыр         Лепешка Узбекская 330г Тандыр
Печень говяжья п/ф         Печень говяжья п/ф
Лак д/ногтей с Кальцием Classics 11мл         Лак д/ногтей с Кальцием Classics 11мл
1ШТ OB З/Щ КОМП ПЯТИСТОР 40 СР         1ШТ OB З/Щ КОМП ПЯТИСТАХ 40 СР
Вода газ Кока-Кола 1л пэт 1/12         Вода газ Кока-Кола 1л пэт 1/12
ПЕРЧАТКИ "ЛАТЕКО"         ПЕРЧАТКИ "ЛАТЕКО"
УКРОП ЛОТОК 50 ГРАММ         УКРОП ЛОТОК 50 ГРАММ
ФРУКТЫ СВ. ГРУША ПЭКХЭМ 1КГ         ФРУКТЫ СВ. ГРУША БЕКХЭМ 1КГ
Помидор красный КНР         Помидор красный КНР
Чипсы Dr.KORNER кукурузно-рисо         Чипсы Dr.KORNER кукурузно-риск
Гель Sanita Антиржавчина 500г СХЗ         Гель Sanita Антиржавчина 500г СЭЗ
Икра кабачковая С&В 680г. СТМ Рус         Икра кабачковая С&В 680г. СТМ Рус
Кофе  NESCAFE

...

Соломка Пеперо в шок глазури/арахис 36г Лотте         Поломка Период в шок глазури/арахис 36г Лотте
СВЕКЛА ОТВАРНАЯ         СВЕКЛА ОТВАРНАЯ
Фарш тел охл 400г ГЗМС Розовая телятина Мираторг Россия         Фарш тел охл 400г ГИМС Розовая телятина Мираторг Россия
 СЕЛЯН.Яйцо перепелиное ст.20шт          СЕЛЯН.Яйцо перепелиное ст.20шт
БАГЕТ С ВЕТЧИНОЙ И СЫРОМ 270 Г ИП СТРЕКАЛОВСКАЯ         БАГЕТ С ВЕТЧИНОЙ И СЫРОМ 270 Г ИП СТРЕКАЛОВСКАЯ
Чай Гринфилд Саммер Боукет напиток 25пак         Чай Гринфилд Саммер Букет напиток 25пак
Смесь карт по-деревен в чесноч соусе Knorr 28г м/у         Смесь карт по-деревне в чесноч соусе Knorr 28г м/у
Кесадилья с грибами         Кесадилья с грибами
Товар 49         Товар 49
ДОБРЫЙ 0.11Л СМУЗЗИ         ДОБРЫЙ 0.11Л МУЗЫКИ
ТВОР.ДЕСЕРТ 130Г5,4%         ТВОР.ДЕСЕРТ 130Г5,4%
Жевательная резинка Орбит XXL Баблм         Жевательная резинка Орбит XXL Баблм
Сосиски Баварские 0,420кг/Стародворье         Сосиски Баварские 0,420кг/Стародворье
3) Салат из говядины и жарены

Крылья Акашево охл. подложка вес. 1 кг         Крылья Краевой охл. подложка вес. 1 кг
Биотворог ТЕМА банан 4.2% 100г         Биотворог ТЕМА банан 4.2% 100г
КД/ПОДУШ.С ШОК.НАЧ.2         КД/ПОДУШ.С ШОК.НАЧ.2
Печень трески по-приморски 200г ККР и ПП         Печень трески по-приморский 200г КНР и ПП
Раф с сиропом 250 мл         Раф с сиропом 250 мл
Прокладки Натурелла ультра нормал 10шт         Прокладки Натурелла ультра норма 10шт
Ветчина ИНДЕЙКИ п/а 500г Атяше         Ветчина ИНДЕЙКИ п/а 500г Атяше
MARTINI BIANCO 1.5L             Вермут Италия         MARTINI BIANCO 1.5L             Вермут Италия
Филe бедра цыпленка Просто Азб         Филe бедра цыпленка Просто Азс
Арбуз Иран         Арбуз Иран
Пирожное Ореховое с пралине от         Пирожное Ореховое с пралине от
Филе лосося с жульеном 1 порц.         Филе лосося с жульеном 1 пор.
энтерол капс. 250мг n30         интером касс. 250мг n30
Инструменты "РАЗНОЕ" актуальное         Инструменты "РАЗНОЕ" актуальное
БЗМЖ Молоко деревен. отборное 3

1 X Прием (осмотр, консультация) уро         1 X Прием (осмотр, консультация) уро
Арахисовая паста классическая,шт         Арахисовая паста классическая,шт
БАТОНЧИК 82Г         БАТОНЧИК 82Г
СЫР МОЦАРЕЛЛА ПИЦЦ         СЫР МОЦАРЕЛЛА ПИЦЦУ
фурадонин авексима таб 50мг n20         фурадонин максима таб 50мг n20
Ризотто RISO SCOTTI с белыми г         Ризотто RISO SCOTTI с белыми г
Компот 0,5 (бутылка) шт         Компот 0,5 (бутылка) шт
1) Чай с лимоном         1) Чай с лимоном
Филе куриное с томатами 90 гр.         Филе куриное с томатами 90 гр.
Торты         Торты
Пиво Балтика №7 0,45л ж/б 5,4% (24)_J_Балтика Ярославль         Пиво Балтика №7 0,45л ж/б 5,4% (24)_J_Балтика Ярославль
КРУПА ПМЖЛ КУКУРУЗНАЯ ШЛИФОВАННАЯ $4 700Г         КРУПА ПМЖЛ КУКУРУЗА ШЛИФОВАННАЯ $4 700Г
1/1 Лосьон д/лица га 103мл         1/1 Лосьон д/лица га 103мл
Конфеты Европомадка 1кг сгущ/топл мол/сливки Ламзурь         Конфеты Европомадка 1кг сгущ/топ мол/сливки Лазурь
Шоколад Альпен Гольд 90г арахис/кукур хлопья Монде

 Мол.ул"БожКор"3,2%900/950млТБА          Мол.ул"БожКор"3,2%900/950маРТА
 Яйцо курС2 фас 10шт бокс Город          Яйцо курС2 фас 10шт бокс Город
НАР.КОЛБ.ДОКТ.190Г         НАР.КОЛБ.ДОКТ.190Г
70Г ЧИПСЫ МК СОЛЬ         70Г ЧИПСЫ МК СОЛЬ
Кукуруза Лорадо 425мл сладкая ж/б         Кукуруза Лорадо 425мл сладкая ж/б
 Газ.нап"Пепси"0,5л          Газ.нап"Пепси"0,5л
Тарелки одноразовые Лимон 24 с         Тарелки одноразовые Лимон 24 с
Чай TESS Flirt зеленый аром пе         Чай TESS Flirt зеленый даром пе
Губка бытовая с чистящим слоем 10 шт         Губка бытовая с чистящим слоем 10 шт
КРУПА ПМЖЛ ПЕРЛОВАЯ ЯЧМЕННАЯ НОМЕР ПЕРВЫЙ 800Г         КРУПА ПМЖЛ ПЕРЛОВАЯ КАМЕННАЯ НОМЕР ПЕРВЫЙ 800Г
Тайский салат с баклажанами         Тайский салат с баклажанами
* 48 КОП.Мор.ПЛОМБ.мал.в.в/ст.91г         * 48 КОП.Мор.ПЛОМБ.мал.в.в/ст.91г
КД/ЧЕСНОК 10ГР         КД/ЧЕСНОК 10ГР
Облепиховое масло супп рект 500мг №10         Облепиховое масло суп реки 500мг №10
Пирожок с яблоками 90гр         Пирожок с яблоками 90гр


KeyboardInterrupt: 

In [1]:
import jamspell

In [10]:
corrector = jamspell.TSpellCorrector()

In [2]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('ru_small.bin')

True

In [6]:
corrector.GetCandidates(['я', 'русский', 'человк', 'по', 'происхождению'], 1)

('русский',
 'русской',
 'русских',
 'русские',
 'русским',
 'русски',
 'прусский',
 'румский',
 'рузский')

In [12]:
corrector.FixFragment('я просто невероянт зочу спыть')

'я просто невероятно хочу спать'

## I. Предобработка текста - внимание к мелочам

Стоит понимать, что при работе с чеками мы работаем не с обычным текстом, а с ужасно зашумлённым и непредобработанным. Поюсню это на основных моментах, которые учитывались при предобработке текста:

1. Достаём слова из текста

Опишем возможные ситуации, которые встречаются в тексте.

- неразделенные слова, например: `КормДляКошек` (*корм для кошек*) или `презервативыdurex` (*презервативы durex*)
- неразделенные слова, но с опечатками/сокращениями внутри, например: `кормдкошек` (*корм для кошек*) - необходимо построить алгоритм поиска опечаток в словах
- кстати про опечатки, они могут быть и в одиночных словах, так что это тоже стоит учитывать
- всевозможные сокращения одного и того же слова, например, слово шампунь в чеках сокращалось 6-ую различными способами:
`ш-нь, ш/нь, шамп., шампн., шампун., шмпнь` и это, наверное, не всё. Так же сокращения нужно применять тоже осторожно, полагаясь на контекст, потому что `шамп` может оказаться и *шампанским*. Таким образом необходимо построить алгоритм по сокращенному слову пытаться предсказывать его нормальную форму.
- слова могут применяться по-разному в разных контекстах, например слово *краска* применялось в нескольких категориях - строительные материалы, рисование, цветы
- английские слова часто могут быть подсказкой для определенной категории, поэтому их стоит хотя бы не выбрасывать: `например`, в `101` категории, представляющая из себя спортивное питание, было несколько компаний, которые связаны только со спортивным питанием
- стоит отметить, что кажущиеся бессмысленными некоторые слова могут являться *аббревиатурой*, из которой можно много понять о категории чека. Например, в `92` категории есть очень много таких слов, например `хвс`: `холодное водоснабжение`, `жбо` - жидко бытовые отходы;
- еще важно при парсинге слов искать различные сокращения вида `<буква>/<буква>`, для примера `б/а` - *безалкогольный*, а `х/к` - холодное копчение и.т.д - таких слов очень много и их тоже надо пытаться как-то понимать.
- для трудно разделимых категорий можно попытаться понять общий смысл у слов в этой категории и попытаться добавить ключевое слово к каждому встречающемуся. Так, например, `90` категория была представлена очень малым количеством уникальных чеков, однако из слов `выходной, боулинг, бенгальский, батут, дискотека` можно было понять, что речь идет о каком-то отдыхе или празднике и добавить в список слов `праздник`, `удовольствие`, что позволяет лучше разделять классы между собой

- при парсинге в лоб легко упустить какие-то мелочи: так, например в `7` и `9` категориях были представлены лишь какие-то двухбуквенные слова в перемешку с цифрами, по типу `АИ-92-k5` и при обработке текста в данной строке не найдется ничего полезного. Однако информации здесь очень много: данная абревиатура свойственна заправочным станциям, которые продают (евро) топливо на газовых голоках. Такие ситуации можно решать с помощью `regex` шаблонов: 

>  ```python
import re
s = "[аи|а|a|aи]{1,}[- ][0-9]{2,}[\w\/]{1,}[- ][0-9]{2,}[- ]\w\d|
    [\w\/]{1,}[- ][0-9]{2,}-\w\d|
    [аи|а|a|aи]{1,}[-+ | 0-9]{2,}[0-9a-zа-я]{2,}|
    [аи|а|a|aи]{1,}[0-9]{2,}-\w\d|[аи|а|a|aи]{1,}[0-9-]{2,}|
    трк|бензин|"
words = re.findall(r'{}'.format(s), z)```

- некоторые слова сокращаются до одной буквы - это было очень неприятно.

В итоге данные идеи могли помочь в предобработке текста и могут быть взяты на вооржуение в будущем. Задача расстановки пробелов или исправления опечаток в зашумлённых текстах - интересная и важная задача.

2. Знаки препинания

    - в чеках в этом плане творилась полная неразбериха. Слова могли сокращаться таким образом `стекл.очист. 200мл.д/т` и приходилось использовать какие-то хитрые сплиты - создавать шаблоны для сокращений `<word>/<word>`, проверять не сокращённое ли слово и.т.д
    - иногда было важно то, что стоит в кавычках `"<word>"` или скобках `"(word)"`, это тоже стоит учитывать
    
3. Цифры

    - с цифрами у меня был диалог короткий - я выуживал лишь наиболее встречающиеся конструкции, такие как `\d+л|мл|ml` или та же самая конструкция, но с килограммами, милиграммами и их сокращениями
    
    
В итоге собирался словарь сокращений и писалась по ходу огромная и очень коряво-написанная функция для преобразования текста с огромным количеством `regex` выражений и условий. Но главное было не в реализации, а в идеях - что можно выжать из текста. Для любителей кода я в конце ноутбука помещу его, но я заранее извиняюсь :)

Примеры предобработки текста:


    - INPUT: Спинки варено-копченые из мяса ЦБ в/у охладенные

    - OUPTUT: спинки варено копченые из мяса цыпленок бройлер вакуумная упаковка охладенные

## II. Классы товаров

Каждый класс товара я просматривал отдельно для каждой категории, находя какие-то смысловые общие нагрузки или заменяя опечатки, которые нельзя заменить программно. Таким образом, спустя несколько дней обработки, я получил следующую таблицу категорий.

- `0` - алкогольная продукция
- `1` - презерватиры
- `2` - сигареты
- `3` - стики, нагреваемые табачные палочки
- `4` - автомобильная лампа
- `6` - различные приборы чистки для авто: щётки, стеклоочистители
- `7` - газовые колонки, топливо, заправка
- `9` - дизельное топливо
- `11` - щётка, услуги автомойки
- `12` - масла, смазки, ароматизаторы, антифризы для автомобилей
- `13` - ремонт машин
- `19` - канцелярские принадлежности, бумага
- `20` - газеты, журналы
- `24` - детская литература, книжки для малышки
- `26` - органайзеры, ножницы, листки, закладки, корректоры
- `27` - обложки, вкладыши, папки
- `29` - карандаши, ручки
- `30` - тетради, дошкольные предметы, прописи
- `31` - все для художников
- `35` - эфирные масла и свечи
- `36` - витамины, гематогены, капсулы
- `37` - детские: подгузники, мыло детское, шампуни
- `38` - таблетки
- `39` - контактные линзы
- `40` - туалетная бумага, зубная щетка, паста, тампоны
- `41` - тональные крема, тушь, помада
- `42` - лак для ногтей, маникюр, гель
- `43` - тесты на беременность, шприцы, бахилы, бинты, катетеры
- `45` - врач: узи, кт, зубной
- `46` - парфюмерия, косметика, туалетная вода:
- `49` - все для волос: шампуни, расчесчки.
- `50` - различные маски, крема: для лица
- `51` - мыла, масла, дезодорант: для тела
- `52` - шапки, варежки, ремни...
- `53` - просто брюки
- `54` - куртки, плащи, фуфайки..
- `55` - только джинсы 
- `56` - брюки, сорочки, шорты
- `57` - обувь: туфли, ботинки, сандали, тапочки
- `58` - платье + сарафан
- `60` - толстовки, джемперы, кардиганы, кофты
- `61` - колготки, майки, куча одежды
- `62` - спортивная одежда, шорты, треники, перекрывается...
- `66` - пицца 
- `67` - роллы, суши..
- `68` - бургеры, сэндвичи, бутреброды..
- `69` - байтсы, куриные ножки, чикенбургеры - видимо, курица и.т.д.
- `70` - кофе их разновидности, чай, какао
- `71` - рестораны продукты: можно парсить названия по составу продуктов: всевозможные продукты!!
- `72` - детское питание: фруто няня, соки для детей, пюрешки
- `73` - замороженная продукция: рыба, замороженное мясо, тесто, пельмени
- `74` - колбаса, свинина, сервелаты
- `75` - консервно баночная продукция: ананасы в банках, шпроты, соленья
- `76` - крупы: макароны, греча, каши
- `77` - соусы, кетчупы, масло подсолнечное
- `78` - молочные продукты: сыр, молоко, творог и.т.д
- `79` - вот тут уже мясо: шницели, языки, курица, стейки
- `80` - овощи и фрукты
- `81` - орехи, чипсы, сухарики, гренки, арахис + кукуруза, горошек
- `82` - рыбы... очень много различной рыбы
- `83` - напитки, соки
- `84` - наиболее представительная категория - сладости, ватрушки, хлебобулочные изделия, шоколадки: очень много
- `85` - чаи кофе - видимо, нужно смотреть на цену, потому что будет очень много пересечений с категорией `70` - абсолютно идентичные. Видимо, данная категория: *кофе в магазине*, а `70` - в *ресторанах*.
- `90` - праздничные товары, веселье, хлопушки, различные удовольствия
- `92` - `ЖКХ` услуги
- `96` - мячи насосы
- `97` - бассейн, билеты в бассейн, маски продажа, очки для плавания, прокат катамаранов, добавил данные
- `100` - наколенники, шлема - защитные устройства для спорта
- `101` - протеиновые батончики, набор массы, смеси
- `102` - спортивное оборудование: `штанги, гантели`. добавил материалы
- `103` - все для рыбалки, походов, палатки, 
- `105` - различные трубы, Отводки, муфты, прокладки, связанные с водой
- `106` - дверные замки, петли, застежки дверей
- `107` - плоскогупцы, шпатели, отвертки, напильники, малярные инструменты,  лобзики: строительные красящие
- `108` - аэрозоли, растворители, анисептикик - смеси
- `109` - гайки, болты, гвозди
- `111` - сантехника водные:  фильтры для раковин и.т.д
- `114` - суперклеи, изолента, наждачная бумага, герметик, изоляция, клеи, клейкая лента, рубероид
- `115` - тройники, розетки, провода, предохранители, кабеля, выключатели: все, что связано с электричеством
- `117` - освежители, перчатки, стирательные порошки, мыло, пятновывадительные: со стиркой связанные
- `118` - фоторамки, украшения, шары, горшки, свечи - всякие нужные материалы для дома
- `120` - игрушки для детей: `лего`, машинки, конструкторы и.т.д
- `121` - пакет (это не категория, просто единственное слово в этой категории)
- `128` - прожектора, светильники, лампы, светодиоды: все, что со светом связано
- `130` - банки, бутылки, декоративные контейнеры, кружки, стаканчики, крышки, миски, прихватки, салатницы, тарелки: `столовые приборы`
- `133` - полотенца, простыни, скатерти, салфетки, подушки, одеяла, наволочки - хлопчато-бумажные
- `138` - уголь, шампуры, решетки, сетка, щепа, черенки, лопаты, жидкость для розжига
- `139` - мешки для мусора, зажигалки, мешки полиэтиленовые, перчатки - все для уборки + зажигалки(
- `140` - цветы (для посадки), семена цветов, грунт для земли
- `143` - пакет (это не категория, просто единственное слово в этой категории)
- `145` - корм для кошек
- `150` - корм для собак
- `163` - рисование, раскраски
- `164` - пластилин, тесто для лепки, глина
- `167` - все для вышивания, нитки, ленты, бусины, крючки, стразы, ткань
- `177` - для телефона: батарейки, держатели для телефона, чехлы для телефона, штекеры, электронное питание телефона, кабель для телефона lighting usb
- `203` - все, что связано с упаковкой товара, коробки, конверты, упаковочаня бумага, одноразовые упаковки, пакеты упаковочные
- `204` - какие-то услуги, посещения, услуги, гости, акции, но такое ощущение, что это категория для неопределимого.

Да, тут собрано буквально всё, что есть в мире. Весьма различимы несколько кластеров - еда, продукты, одежда, товары для дома, декоративные товары, развлечения. Каждая подгруппа также делится по смыслу на категории, однако встречаются и наложения, которые тяжело отметить даже специалисту - например `204` категория собрала в себя всевозможные услуги, посещения, праздники, встречалась и еда и её было тяжело отделять, как тяжело отделять кофейную продукцию в ресторанах от кофейной продукции в магазинах (категории `70` и `85`). По результатам отмечу следующие наблюдения:

- все категории, в которых было большое количество уникальных чеков (больше 1000) (кроме `204`), отлично отделялись
- были и категории с небольшим количеством наблюдений, но с отличной разделимостью благодаря большому количество одного повторящегося у всей категории слова - например `джинсы` или *прерзервативы*.
- а вот категории с маленьким количеством уникальных чеков и без отличительной какой-нибудь черты (достаточно было одного повторяющегося слова) разделялись плохо - были предприняты даже попытки разметить и добавить в недостающие категории товары, но это стало непосильной задачей.

## III. `Использованные модели`

В плане выбора модели я не блистал оригинальностью, перепробоав всевозможные типичные методы, но приведу те, которые мне понравились по идее и/или результату. Перед применением данных моделей производилась предварительная предобработка текста, описанная в пункте `I`:

1. (Идея) 

    - *TRAIN*: посчитать для каждой категории с помощью `Counter` из бибилотеки `collections` встречаемость каждого слова в определенной категории, создать словарь, в котором ключи - номер категории, а значения - словарь `counter` наиболее встречаемых слов в данном словаре (например, я брал `n=50`. Частоты нормируются и взвешиваются пропорциально встречаемости категории в тренировчном множестве
    
    - *TEST*: предобработать аналогично тренировочному множеству тестовойе множество, посчитать вероятность принадлежности определенной категории через суммму весов - взять категорию с наибольшей суммой весов
    
Иногда в соревнованиях выстреливали простые модели, но здесь результат был ужасным - лучший `0.6`.

2. (Идея) на предобработанных лемматезированных текстах я подумал, что можно обучить известную модель `Word2Vec`, только на всех `8` миллионах чеках уникальных чеках, а затем построить эмбеддинги предложений как среднее эмбеддингов и числа слов, которые есть из предложения в эмбеддинге. Получились даже вполне хорошие эмбеддинги (код я оставлю внизу, чтобы можно было их пощупать, есть разделимость по категориям, но результат разочаровал - `KNN CLassifier 9_neighbours` с метрикой `cosine` - лучший результат `0.77`.

3. (Результат) Предобработка текста + добавление цифр + получение прзнаков `TFIDF` со следующими параметрами:

```python
tfidf = TfidfVectorizer(
    max_features=800000, 
    ngram_range=(1, 6), 
    stop_words ='russian',
    analyzer="char_wb", 
    norm = 'l2'
)```

Над всем этим используется линейный `SVMCLassifier` и получаем результат - `0.844`. Обидно, когда не придумал нового.

4. Были ещё и нейронные рекурретные сети, но лучше о них я рассказывать не буду...

## IV. Выводы

Думаю, что по данному соревнованию я сделаю следующие выводы и наблюдения:

    - предобработка текста очень важна и ею нужно заниматься (а ещё это очень прокачивает знание регулярных выражений)
    - разметка малопредставленных категорий помогла бы улучшить качество модели
    - в данной работе совершенно не использовалась информация о дне недели, цене, времени, поэтому хотелось бы в дальнейшем научиться строить модели, учитывая дополнительные признаки.

Спасибо за соревнование, буду продолжать работать над идеями. А сейчас для дождавшихся - код :)

## V Дополнительно: код

Код для предобработки.

!pip install pymorphy2

In [39]:
import pymorphy2
from functools import lru_cache
ru_lemmatizer = pymorphy2.MorphAnalyzer()
@lru_cache(maxsize=10 ** 6)
def lru_lemmatizer(word):
    '''лемматизация'''
    return ru_lemmatizer.parse(word)[0].normal_form

In [40]:
def prepare(train_data, perevod, lemir, maslin, prazdnik, do_lemma=False):
    '''
    INPUT
    
    ---------
    
    train_data :: pd.Dataframe - исходные данные, необходимо, чтобы был столбец item_name
    perevod :: словарь сокращений
    lemir :: список принадлежности замороженным продуктам
    maslin :: список принадлежности категории масляных банок и консервов
    prazdnik :: список слов, подразумевающих отдых или праздник
    do_lemma :: default=False, лемматизация слов в конце с помощью PyMorhy
    
    OUPTUT:
    
    vocab_large_dict :: словарь, ключи - уникальные значения train_data['item_name'], 
                                 значения - обработанная строка слов, слова отделены одиночным пробелом
                                 
    train_data :: новый DataFrame со столбцом preprocess
    
    '''
    un_values = train_data['item_name'].value_counts()
    print('STEP 1')
    a = []
    wd = {}
    for i in list(un_values.keys()):
        new = i.lower()
        splitted = re.findall(r'(\b[a-zA-Zа-яА-ЯеЁ_]+\b)', new)
        if splitted != []:
            if splitted[0] == new:
                if i.isupper():
                    #print(i)
                    wd[i] = {}
                    wd[i]['done'] = 'maybe'
                    wd[i]['word'] = i.lower()
                elif len(re.split("(?=[А-ЯA-Z\_])", i)) >= 2:
                    split = re.split("(?=[А-ЯA-Z\_])", i)[1:]
                    if any([x.lower() for x in split if len(x) == 1 and x not in string.punctuation]):
                        if ''.join(split[-3:]) == 'ИМП':
                            wd[i] = {}
                            wd[i]['done'] = 'yes'
                            wd[i]['word'] = ' '.join([x.lower() for x in split if len(x) !=1] + ['ИМП'])
                        else:
                            wd[i] = {}
                            wd[i]['done'] = 'no'
                            wd[i]['word'] = ' '.join([x.lower() for x in split if len(x) !=1])
                    else: 
                        wd[i] = {}
                        wd[i]['done'] = 'yes'
                        wd[i]['word'] = ' '.join([x.lower() for x in split if x not in string.punctuation])
                else:
                    a.append(i)
            else:
                a.append(i)
        else:
            a.append(i)
    print('STEP 2')
    b = []
    wd_2 = {}
    for i in a:
        text = re.sub(r'[{}]'.format(string.punctuation), ' ', i)
        splitted = re.findall(r'(\b[a-zA-Zа-яА-ЯеЁ_]+\b)', text)
        sub3 = re.sub(r'(\b[a-zA-Zа-яА-ЯеЁ_]+\b)', ' ', text)
        if ' '.join(splitted) == i:
            new_word = []
            for j in splitted:
                if j in perevod.keys():
                    new_word.append(perevod[j])
                elif j.isupper():
                    new_word.append(j.lower())
                elif j.islower():
                    new_word.append(j.lower())
                else:
                    split = re.split("(?=[А-ЯA-Z\_])", j)[1:]
                    if split is not []:
                        for x in split:
                            new_word.append(x.lower())
            #new_word
            wd_2[i] = {}
            wd_2[i]['done'] = 'maybe'
            wd_2[i]['word'] = ' '.join(new_word)
        else:
            b.append(i)
    print('STEP 3')
    slahes = {}
    wd_3 = {}
    c = []
    for i in b:
        z = text.lower()
        text = i.lower()
        text = re.sub(r'[{}]'.format(''.join([x for x in string.punctuation if x not in ['/', '.', '-', "'"]])), ' ', text)
        words = re.findall(r'\b[a-zA-Zа-яА-ЯеЁ]+\b', text)
        new_word = re.findall(r'\b[a-za-я]{1}/[a-za-я]{1}\b', text)
        sub3 = re.sub(r'\b[a-zA-Zа-яА-ЯеЁ]+\b', ' ', text)
        if new_word != []:
            for j in new_word:
                if j in slahes.keys():
                    slahes[j].append(text)
                else:
                    slahes[j] = []
        if ' '.join(words) == text:
            wd_3[i] = {}
            wd_3[i]['done'] = 'maybe'
            for u in new_word:
                if u in perevod.keys():
                    if any([u == x for x in ['т/в', 'п/в', 'пв', 'тв', 'vsop']]) and any([x in z for x in ['ml', 'м', 'мл', 'одек', 
                                                                                                           'одеколон', 'bruno', 'tentation', 
                                                                                                           'gracia', 'milady']]):
                        if any([u == x for x in ['т/в', 'тв', 'vsop']]):
                            words.append('туалетная вода')
                        else:
                            words.append('парфюмерная вода')
                    else:
                        words.append(perevod[l])
                else:
                    words.append(l)
            wd_3[i]['word'] = ' '.join(words)
        else:
            c.append(i)  
    print('STEP 4')
    wd_4 = {}
    num = 0
    step4 = []
    for i in c:
        z = i.lower()
        text = re.sub(r'[{}]'.format(string.punctuation), ' ', z)
        words = re.findall(r'\b[a-zA-Zа-яА-ЯеЁ]+\b', text)
        if re.search('\d+', z) is None:
            if ' '.join(words) == text:
                w = re.split('[ ,]', z)
                words = []
                for u in w:


                    if re.search(r'\w{2,}[-]\w{2,}|\w{3,}[.]\w{3,}|\w{2,}\/\w{2,}', u) is not None:
                        if u in perevod.keys():
                            words.append(perevod[u])
                        else:
                            for l in re.split('[-.\/]', u):
                                if l in perevod.keys():
                                    words.append(perevod[l])
                                else:
                                    words.append(l)
                    else:
                        if u in perevod.keys():
                            words.append(perevod[u])

                        elif re.search(r'\bд/[a-zA-Zа-яА-ЯеЁ]+\b', u) is not None:
                            words.append('для ' + u.split('/')[-1])
                        else:
                            words.append(u)
                wd_4[i] = {}
                wd_4[i]['done'] = 'maybe'
                wd_4[i]['word'] = ' '.join(words)
                num +=1
            else:
                step4.append(i)
        else:
            step4.append(i)
    print('STEP 5')
    num = 0
    step5 = []
    wd_5 = {}
    for i in step4:
        z = i.lower()
        # text = re.sub(r'[{}]'.format(string.punctuation), ' ', z)
        words = re.findall(r'\b[a-zA-Zа-яА-ЯеЁ]+\b', z)
        if re.search('\d+', z) is None:
            w = re.split('[\(\), .]', z)
            words = []
            for u in w:
                if re.search(r'\w{2,}[-]\w{2,}|\w{3,}[.]\w{3,}|\w{2,}\/\w{2,}|\w{2,}[.,\/]|\(\w{2,}\)|\(\w{2,} \w{2,}\)', u) is not None:

                    if u in perevod.keys():
                        if any([u == x for x in ['т/в', 'п/в', 'пв', 'тв', 'vsop']]) and any([x in z for x in ['ml', 'м', 'мл', 'одек', 
                                                                                                           'одеколон', 'bruno', 'tentation', 
                                                                                                           'gracia', 'milady']]):
                            if any([u == x for x in ['т/в', 'тв']]):
                                words.append('туалетная вода')
                            else:
                                words.append('парфюмерная вода')
                        else:
                            words.append(perevod[u])
                    else:
                        for l in re.split('[-.\/\(\)\\" ]', u):
                            if l in perevod.keys():
                                if any([l == x for x in ['т/в', 'п/в', 'пв', 'тв', 'vsop']]) and any([x in z for x in ['ml', 'м', 'мл', 'одек', 
                                                                                                           'одеколон', 'bruno', 'tentation', 
                                                                                                           'gracia', 'milady']]):
                                    if any([l == x for x in ['т/в', 'тв']]):
                                        words.append('туалетная вода')
                                    else:
                                        words.append('парфюмерная вода')
                                else:
                                    words.append(perevod[l])
                            else:
                                words.append(l)
                else:
                    if u in perevod.keys():
                        if any([u == x for x in ['т/в', 'п/в', 'пв', 'тв', 'vsop']]) and any([x in z for x in ['ml', 'м', 'мл', 'одек', 
                                                                                                           'одеколон', 'bruno', 'tentation', 
                                                                                                           'gracia', 'milady']]):
                            if any([u == x for x in ['т/в', 'тв']]):
                                words.append('туалетная вода')
                            else:
                                words.append('парфюмерная вода')
                        else:
                            words.append(perevod[u])

                    elif re.search(r'\bд/[a-zA-Zа-яА-ЯеЁ]+\b', u) is not None:
                        words.append('для ' + u.split('/')[-1])

                    elif len(u) == 1:
                        None
                    else:
                        words.append(u)
            wd_5[i] = {}
            wd_5[i]['done'] = 'maybe'
            text = re.sub(r'[{}]'.format(string.punctuation), ' ', ' '.join(words))
            if text.split() != []:
                if text.split()[0].endswith('илин'):
                    wd_5[i]['word'] = text.split()[0]
                else:
                    wd_5[i]['word'] = ' '.join([x for x in text.split()])
            else:
                wd_5[i]['word'] = i
            num +=1
        else:
            step5.append(i)
    print('STEP 6')
    
    sum_dict = {   
    'аи': 'заправка автотранспорт топливо евро газовая колонка', 
    'k5': 'заправка безопасное топливо евро газовая колонка', 
        'к5': 'заправка безопасное топливо евро газовая колонка',
    'а': 'заправка автотранспорт топливо евро газовая колонка', 
    '92': 'заправка бензин евро газовая колонка', 
    '95': 'заправка бензин евро газовая колонка', 
    '98': 'заправка бензин евро газовая колонка',
    '95е': 'заправка бензин евро газовая колонка', 
    'аи92': 'заправка автотранспорт топливо бензин евро газовая колонка', 
    'трк': 'заправка топливо газовая колонка', 
        'бензин': 'бензин', 
        'автомобиль': 'автотранспорт', 
        'джет': 'джет', 
        'премимум': 'премиум', 
        'класс': 'класс', 
        'колонка': 'колонка'}
    
    sum_dict_category_9 = {'диз': 'дизельное', 
                       'топ': 'топливо', 'дт': 'дизельное топливо', 
                       'колонка': 'колонка', 'дизельное': 'дизельное', 
                       'суг': 'сниженные углеводороные газы дизельное топливо', 
                       'танеко': 'производитель дизельное топливо', 
                       'пбт': 'пропан бутан технический дизельное топливо', 
                       'k5': 'заправка дизельное топливо', 'е': 'межсезонное', 
                       'л': 'летнее', 'з': 'зимнее', 
                       'еврос': 'европейский', '92': 'вид топливо', 'к5': 'заправка дизельное топливо', 
                      'трк': 'заправка', 'tpk': 'заправка'}
    
    num = 0
    step6 = []
    wd_6 = {}
   
    for i in step5:
        z = i.lower()
        text = re.sub('\d+', ' ', z)
        #words = re.findall(r'\b[a-zA-Zа-яА-ЯеЁ]+\b', text)
        words = []
        s = "[аи|а|a|aи]{1,}[- ][0-9]{2,}[\w\/]{1,}[- ][0-9]{2,}[- ]\w\d|[\w\/]{1,}[- ][0-9]{2,}-\w\d|[аи|а|a|aи]{1,}[0-9]{2,}-\w\d|[аи|а|a|aи]{1,}[0-9-]{2,}"
        if len(re.findall(r'{}'.format(s), z)) != 0:
            s = "[аи|а|a|aи]{1,}[- ][0-9]{2,}[\w\/]{1,}[- ][0-9]{2,}[- ]\w\d|[\w\/]{1,}[- ][0-9]{2,}-\w\d|[аи|а|a|aи]{1,}[-+ | 0-9]{2,}[0-9a-zа-я]{2,}|[аи|а|a|aи]{1,}[0-9]{2,}-\w\d|[аи|а|a|aи]{1,}[0-9-]{2,}|трк|бензин|автомобильный|джет|премиум|класс|колонка|95"
            new_wo = re.findall(r'{}'.format(s), z)
            
            for we in new_wo:
                my = re.split('[\(\), /+-]', we)
                for m in my:
                    if m in sum_dict.keys():
                        words.append(sum_dict[m])
            wd_6[i] = {}
            wd_6[i]['done'] = 'maybe'
            wd_6[i]['word'] = ' '.join(words)
            
        
        elif len(re.findall(r'{}'.format("дт[_ -]\w{1,}[_ -]\w\d|дт[_ -]\w{1,}|суг|пбт|трк|tpk|танеко"), z)) != 0:
            s = "дт[_ -]\w{1,}[_ -]\w\d|дт[_ -]\w{1,}|трк|tpk|92|диз|топливо|дт|колонка|топ|дизельное|суг|танеко|пбт"
            new_wo = re.findall(r'{}'.format(s), z)
            for we in new_wo:
                my = re.split('[\(\), /+-]', we)
                for m in my:
                    if m in sum_dict_category_9.keys():
                        words.append(sum_dict_category_9[m])
            wd_6[i] = {}
            wd_6[i]['done'] = 'maybe'
            wd_6[i]['word'] = ' '.join(words)
        
        elif re.search('\d+', text) is None:
            w = re.split('[\(\), .]', text)
            for u in w:
                if re.search(r'\w{2,}[-]\w{2,}|\w{3,}[.]\w{3,}|\w{2,}\/\w{2,}|\w{2,}[.,\/]|\(\w{2,}\)|\(\w{2,} \w{2,}\)', u) is not None:

                    if u in perevod.keys():
                        words.append(perevod[u])
                    else:
                        for l in re.split('[-.\/\(\)\\" ]', u):
                            if l in perevod.keys():
                                if any([l == x for x in ['т/в', 'п/в', 'пв', 'тв', 'vsop']]) and any([x in z for x in ['ml', 'м', 'мл', 'одек', 
                                                                                                           'одеколон', 'bruno', 'tentation', 
                                                                                                           'gracia', 'milady']]):
                                    if any([l == x for x in ['т/в', 'тв', 'vsop']]):
                                        words.append('туалетная вода')
                                    else:
                                        words.append('парфюмерная вода')
                                else:
                                    words.append(perevod[l])
                            else:
                                words.append(l)
                else:
                    if any([u == x for x in ['т/в', 'п/в', 'пв', 'тв', 'vsop']]) and any([x in z for x in ['ml', 'м', 'мл', 'одек', 
                                                                                                           'одеколон', 'bruno', 'tentation', 
                                                                                                           'gracia', 'milady']]):
                        if any([u == x for x in ['т/в', 'тв', 'vsop']]):
                            words.append('туалетная вода')
                        else:
                            words.append('парфюмерная вода')
                    elif u in perevod.keys():
                            words.append(perevod[u])
                    elif re.search(r'\bд/[a-zA-Zа-яА-ЯеЁ]+\b', u) is not None:
                        words.append('для ' + u.split('/')[-1])

                    elif len(u) == 1:
                        None
                    else:
                        words.append(u)
            wd_6[i] = {}
            wd_6[i]['done'] = 'maybe'
            text = re.sub(r'[{}]'.format(string.punctuation), ' ', ' '.join(words))
            if text.split() != []:
                if text.split()[0].endswith('илин'):
                    wd_6[i]['word'] = text.split()[0]
                else:
                    wd_6[i]['word'] = ' '.join([x for x in text.split()])
            else:
                wd_6[i]['word'] = ' '.join(words)
            num +=1
        else:
            step6.append(i)
    LARGE_DICT = {}
    for i in [wd, wd_2, wd_3, wd_4, wd_5, wd_6]:
        LARGE_DICT.update(i)
    vocab_large_dict = {k: ' '.join([' '.join((re.sub(r'[{}]'.format(string.punctuation + '№'), 
                                            '', 
                                            re.sub('k', 'к', x)).replace('\\', '')).split(' ')) for x in w['word'].split(' ')])
                        for k, w in LARGE_DICT.items()}
    
    
    vocab_large_dict = {k: ' '.join([re.sub('k', 'к', re.sub(r'[{}]'.format(string.punctuation), '', perevod[x] if x in perevod.keys() else x)) for x in w.split(' ')]) for k, w in vocab_large_dict.items()}
    print('MAPPING....')
    vocab_large_dict = {k: ' '.join(['кислота' if x == 'к' and x != w.split(' ')[-1] and w.split(' ')[w.split(' ').index(x) + 1] == 'та' else x for x in w.split(' ') if x != 'та']) for k, w in vocab_large_dict.items()}
    vocab_large_dict = {k: ' '.join(['мороженое' if x == 'м' and x != w.split(' ')[-1] and w.split(' ')[w.split(' ').index(x) + 1] == 'е' else x for x in w.split(' ') if x != 'е']) for k, w in vocab_large_dict.items()}
    vocab_large_dict = {k: ' '.join(['внутреннее применение' if x == 'вн' and x != w.split(' ')[-1] and w.split(' ')[w.split(' ').index(x) + 1] == 'пр' else x for x in w.split(' ') if x != 'пр']) for k, w in vocab_large_dict.items()}
    vocab_large_dict = {k: ' '.join(['раствор' if x == 'р' and x != w.split(' ')[-1] and w.split(' ')[w.split(' ').index(x) + 1] == 'р' else x for x in w.split(' ')]) for k, w in vocab_large_dict.items()}
    vocab_large_dict = {k: ' '.join(['спортивный тренировочный инвентарь силовые тренировки' if x == 'speed' and x != w.split(' ')[-1] and w.split(' ')[w.split(' ').index(x) + 1] == 'bag' else x for x in w.split(' ')]) for k, w in vocab_large_dict.items()}
    vocab_large_dict = {k: ' '.join(['попкорн' if x == 'поп' and x != w.split(' ')[-1] and w.split(' ')[w.split(' ').index(x) + 1] == 'корн' else x for x in w.split(' ') if x != 'корн']) for k, w in vocab_large_dict.items()}
    vocab_large_dict = {k: ' '.join([perevod[x] if x in perevod.keys() else x for x in w.split(' ')]) for k, w in vocab_large_dict.items()}
    vocab_large_dict = {k: ' '.join(['праздник ' + x if x in prazdnik else x for x in w.split(' ')]) for k, w in vocab_large_dict.items()}
    if do_lemma:
        vocab_large_dict = {k: ' '.join([lru_lemmatizer(x) for x in w.split(' ')]) for k, w in vocab_large_dict.items()}
        vocab_large_dict = {k: ' '.join([perevod[x] if x in perevod.keys() else x for x in w.split(' ')]) for k, w in vocab_large_dict.items()}
    vocab_large_dict = {k: ' '.join(['замороженная продукция ' + x if x in lemir else x for x in w.split(' ')]) for k, w in vocab_large_dict.items()}
    vocab_large_dict = {k: ' '.join(['консервная баночная продукция ' + x if x in maslin else x for x in w.split(' ')]) for k, w in vocab_large_dict.items()}
    vocab_large_dict = {k: ' '.join(['праздник ' + x if x in prazdnik else x for x in w.split(' ')]) for k, w in vocab_large_dict.items()}
    train_data['preprocess'] = train_data['item_name'].map(vocab_large_dict)
    return vocab_large_dict, train_data

А далее мой словарь сокращений...

In [41]:
l = set(['эскимо', 'пломбир', 'чебурек', 'чебупели', 'хинкали', 
   'вареник', 'вареники', 'блины', 'блинчики', 'бефстроганов', 'лазанья', 'лед', 'манты',
     'мираторг', 'мороженое',  'пельмени', 'пломбир', 'рожок', 
         'стаканчик',  'булгур', 'сорбет', 
          'пельмени', 'чевапчичи',
     'голубцы',  'брокколи'])
maslin = set(['аджика', 'варенье', 'джем', 'желе', 
             'квашеная', 'консервы', 'лечо', 'маслины', 'мёд', 'оливки', 'паштет', 'сайра', 
              'повидло', 'фасоль', 'шпроты'])

prazdnik = ['выходной', 'боулинг', 'бенгальский', 'аренда', 'пузыри', 'батут', 'шар', 'дискотека', 
           'безлимит', 'аттракцион', 'детский', 'городок', 'огни', 'бар', 'день', 'парк', 'вход', 
            'веселье', 'веселие', 'заезд', 'караоке', 'катание', 'лед', 'лёд', 'льду', 'петарды',
           'катание', 'массовое', 'мыльные', 'радужные', 'хлопушка', 'подарок', 'поздравление', 'баня', 'залпы',
           'свадьба', 'свеча', 'турист', 'мир', 'фейерверки', 'дым', 'фейерверк', 'шар', 'воздушный', 'юбилей', 
           'скидка', 'миллион']

perevod = {'в/с': 'высший сорт', 
           'б/к': 'без костей', 
           'в/у': 'вакуумная упаковка', 
           'м/у': 'мягкая упаковка', 
           'с/к': 'сырокопченая', 
           'ж/б': 'жестяная банка', 
           'п/к': 'полукопченая', 
           'с/м': 'свежемороженый', 
           'ж/р': 'жевательная резинка', 
           'б/г': 'без головы', 
           'б/п': 'большая пачка',
           'п/о': 'пленочная оболочка', 
           'з/п': 'зубная паста', 
           'с/б': 'стеклянная бутылка', 
           'х/к': 'холодное копчение', 
           'ц/о': 'целлофановая оболочка', 
           'б/а': 'безалокогольные', 
           'д/п': 'детское питание',  
           'm/l': 'размер', 
           'з/щ': 'зубная щётка', 
           'п/ф': 'полуфабрикат', 
           'ветч/сыр': 'ветчина сыр', 
           'д/пола': 'для пола', 
           'д/уборки': 'для уборки', 
           'с/с': 'слабо соленая', 
           'охл': 'охлажденная', 
           'д/чист': 'для чистки', 
           'д/колки': 'для колки', 
           'н/р': 'неразделанный', 
           'д/кош': 'для кошки', 
           'туал': 'туалет', 
           'асс-те': 'ассортименте', 
           'пив': 'пивной', 
           'д/м': 'для мужчин', 
           'д/ж': 'для женщин', 
           'д/выпеч': 'для выпечки', 
           'ст': 'стандартный', 
           'стд': 'стандартный', 
           'картоф': 'картофель', 
           'порц': 'порция', 
           'бол': 'большая', 
           'бкомбо': 'большое комбо', 
           'скомбо': 'среднее комбо', 
           'пласт': 'пластиковый', 
           'ср': 'средний', 
           'мал': 'маленький', 
           'табачн': 'табачные', 
           'кокт': 'коктейль', 
           'асс': 'ассортименте', 
           'чл': 'чистая линия', 
           'р-ра': 'раствора', 
           'нап': 'напиток', 
           'энерг': 'энергетический', 
           'ябл': 'яблоко', 
           'фр': 'фрутелла', 
           'автом': 'автомобильная', 
           'шариков': 'шариковых', 
           'мрам': 'мраморной', 
           'гов': 'говядины', 
           'ОСТАН': 'останкинское', 
           'фил': 'филе', 
           'зап': 'запеченное', 
           'прод': 'продукт', 
           'йог': 'йогурт', 
           'устройс': 'устройство', 
           'арб': 'арбуз', 
           'дын': 'дыня', 
           'д/тетрад': 'для тетрадка', 
           'содерж-е': 'содержание', 
           'малин': 'малиновый', 
           'нач': 'начинка', 
           'кукур': 'кукуруза', 
           'карамел': 'карамель', 
           'мясн': 'мясной', 
           'смород': 'смородина', 
           'респуб': 'республика', 
           #'гр': 'грамм', 
           #'кг': 'килограмм', 
           #'шт': 'штук', 'руб': 'рубль', 
           'алког': 'алкоголь', 
           'п/сл': 'полусладкое', 'алк': 'алкоголь', 'фрукт': 'фруктовый', 
           'п/сух': 'полусухое', 'сух': 'сухое', 'кр': 'красное', 'вин': 'вино', 
           'сх': 'сухое', 'красн': 'красное', 'крас': 'красное', 
           'бел/сух': 'белое сухое', 'бут': 'бутылка',
           #'шамп': 'шампанское',
           'шампань': 'шампанское', 
           'шампусик': 'шампанское', 'бел': 'белое', 
           'ж/б': 'железная бутылка', 'б/алк': 'без алкоголь', 
           'б/сух': 'белое сухое', 
           'б': 'белое', 'светл': 'светлое', 
           'балтитка': 'балтика', 'шымкентское': 'шампанское', 
           'п/слад': 'полусладкое', 'жб': 'железная банка',
           'н/ф': 'не фильтрованное', 'н\ф': 'не фильтрованное', 'сл': 'сладкое', 
           'фильт': 'фильтрованное', 'нефильт': 'не фильтрованное', 'жив': 'живое', 
           'п/у': 'прозрачная упаковка', 'алко': 'алкоголь', 'глинтв': 'глинтвейн', 
           #'мл': 'миллиграмм',
           'омсквинпром': 'оксимирон', 
           #'ml': 'миллиграмм', 
           'глинтв.': 'глинтвейн', 'п/с': 'полусладкое', 
           'ж\б': 'железная банка', 'сп': 'спирт', 'спир': 'спирт', 
           'ш-е': 'шампанское', 
           'мягк': 'мягкое', 'н/фил': 'не фильтрованное', 
           'п/уп': 'пластиковая упаковка', 'стар': 'старый', 'бел/сух': 'белый сухой', 'игр': 'игристый', 
           'псх': 'полусухое', 'псл': 'полусладкое', 
           'блсх': 'белое сухое', 'кр/сух': 'красное сухое', 'крсх': 'красное сухое', 'водк': 'водка', 
           'с/а': 'слабо алкогольный', 'слабоалк': 'слабо алкогольный', 
           'слаб': 'слабый', 'слабоал': 'слабо алкогольный', 'ос': 'особая', 
           'бл': 'белое', 'рз': 'розовый', 'игр': 'игристое', 
           'газ': 'газированный', 'водакока': 'водка', 'боч': 'бочонок', 
           'бочон': 'бочонка', 'пивн': 'пивной', 'пив': 'пивной', 
           'през': 'презерватив', 
           'презер': 'презерватив', 'презерв': 'презерватив', 
           'conte': 'contex', 'cont': 'contex', 'дюрекс': 'durex', 
           'сверхтонк': 'сверхтонкие', 'смазк': 'смазка', 'ошибк': 
           'ошибка', 'контекс': 'contex', 'сиг': 'сигареты', 
           'классич': 'классический', 'ассорт': 'ассорти', 
           'сигаретывинстонблю': 'сигареты винстон блю', 
           'сигаретывинстонсильвер': 'сигареты винстон сильвер', 
           'таб': 'табак', 'сигаретылд': 'сигареты', 
           'сигаретытройка': 'сигареты тройка', 
           'сигаретыява': 'сигареты ява', 'l&m': 'LM', 
           'мрц': 'максимально розничная цена', 
           'силвер': 'сильвер', 
           'c-ты': 'сигареты', 'нагр': 'нагреваемые',
           'нагрев': 'нагреваемые', 'табач': 'табак', 
           'жидк': 'жидкий', 'испар': 'испарение', 
           'масл': 'масляный', 'возд': 'воздушный', 
           'торм': 'тормозная', 'рем': 'ремень', 'д/ш': 'для шин', 
           'насад': 'насадка', 'автомоб': 'автомобиль', 
           'с/о': 'стеклоочистителя', 'со': 'стеклоочистителя', 
           'оч': 'стеклоочистителя', 'очист': 'стеклоочистителя', 
           'стеклоочис': 'стеклоочистителя', 
           'стеклооч': 'стеклоочистителя', 'а/м': 'ароматизатор', 
           'вел': 'велосипед', 'бескарк': 'бескаркасная', 'наб': 'набор', 
           'антиф': 'антифриз', 'аром': 'аромат', 'гермет': 'герметик', 'мот': 'моторное', 
           'жид-ть': 'жидкость', 
           'кан': 'канцелярский', 'унив': 'университет', 'тв': 'телевизор', 
           'жур': 'журнал', 'журн': 'журнал', 'ж-л': 'журнал', 'новос': 'новости', 
           'сканв': 'сканворды', 'настоль': 'настольных', 'канц': 'канцелярские', 
           'мелоч': 'мелочей','бейдж': 'бейджик', 'самокл': 'самоклеющаяся', 
           'быстросохн': 'быстро сохнущая', 'клейк': 'клейкая', 'колп': 'колпачок', 
           'цв': 'цвет', 'скоросшиват': 'скоросшиватель', 
           'карт': 'картонный', 'тетр': 'тетрадь', 'свидетел': 'свидетельство', 'заключ': 'заключение', 
           'скоросшив': 'скоросшиватель', 'цветн': 'цветные', 
           'каранд': 'карандаш', 'черн': 'черная', 'ч/г': 'чернографитный', 
           'ч/гр': 'чернографитный', 'платиковый': 'платиковый', 'руч': 'ручка', 
           'гел': 'гелевая', 'чер': 'черная', 'прост': 'простой', 
           'шарик': 'шариковый', 'н/р': 'набор ручек', 'нб': 'набор', 'прозр': 'прозрачный', 'колпач': 'колпачок',
           'гелев': 'гелевая', 'капил': 'капиллярная', 'кап': 'капиллярная', 'син': 'синяя', 'чёрн': 'черная',
          'дневн': 'дневник', 'обл': 'область', 'шк': 'школьная', 'кл': 'класс', 'ежед': 'ежедневник', 
           'ежедн': 'ежедневник', 'синт': 'синтетический', 'кар': 'карандаш', 'клейкарандаш': 'клей карандаш', 
          'рус/яз': 'русский язык', 'кист': 'кисточки', 'раб': 'рабочая', 'накл': 'наклейка', 'ножн': 'ножницы', 
           'днев': 'дневник', 'тет': 'тетрадь', 'универс': 'университет', 'проп': 'пропись', 'дошк': 'школьник', 
           'самоклбумагад': 'самоклеющаяся', 'пап': 'папка', 'отд': 'отделение', 'молн': 'молния', 'глянц': 'глянцевый', 
           'зел': 'зеленый', 'многоцв': 'много цветов', 'заточен': 'заточенный', 'худож': 'художник', 'акв': 'акварель', 
           'леп': 'лепения', 'фломаст': 'фломастер', 'разноцв': 'разноцветный', 'точилкимп': 'точилка', 'раск': 'раскраска',
          'бумаа': 'бумага', 'аромапалочки': 'арома палочки', 'ароматичес': 'ароматический', 'аромнаб': 'арома', 
           'ароматич': 'ароматический', 'эфирн': 'эфирное', 'витамир': 'витамин', 'витаминно': 'витамин', 
           'капс': 'капсулы', 'аевитамины': 'витамины', 'вит': 'витамины', 'жев': 'жевательный', 
           'жеват': 'жевательный', 'к/та': 'кислота', 'обогащ': 'обогащенный', 'вита': 'витамин', 'актив': 'активный', 
          'красногорсклексредства': 'средства', 'стм': 'собственная торговая марка', 'дет': 'детская', 'трубоч': 'трубочка', 
           'подг': 'подгузник', 'подгузн': 'подгузник', 
           'клубн': 'клубничный', 'з/паста': 'зубная паста', 'туалетн': 'туалетное', 'п/заг': 'после загара', 'ручк': 'ручка',
          'влаж': 'влажные', 'экстр': 'экстрат', 'салф': 'салфетка', 'косм': 'косметика', 'пен': 'пена','шам': 'шампунь', 
          'увл': 'увлажнение', 'нат': 'натуральный', 'взр': 'взрослый', 'пел': 'пеленка', 
           'пелен': 'пеленка', 'блест': 'блестящий', 'профилакт': 'профилактика', 'мес': 'месяц', 'сусп': 'суспензия', 
           'наз': 'назальные','назал': 'назальные', 'лексредства': 'лекарство средства', 'увлажняющ': 'увлажняющие', 
           'капс': 'капсулы', 'тбл': 'таблетки', 'водн': 'водный', 'вн/пр': 'внутреннее применение', 'внут': 'внутрь', 
           'табл': 'таблетки', 'наружн': 'наружное', 'прим': 'примение', 'р/ра': 'раствор', 'држ': 'драже', 
          'аэроз': 'аэрозоль', 'аэр': 'аэрозоль', 'к/ты': 'кислота', 'н/ка': 'настойка',  'р-ра': 'раствор',
          'изготовл': 'изготовление', 'пластм': 'пластмассовая', 'оправе': 'оправа', 'конт': 'контактных', 
           'увлажн': 'увлажняющие', 'салфет': 'салфетка', 'auraвлсалф': 'салфетка', 'бум': 'бумага', 
           'влсалфунив': 'салфетка', 'проклад': 'прокладка', 'зп': 'зубная паста', 'полоск': 'полоскание', 
           'антиб': 'антибактериальный', 'ватн': 'ватные', 'вл': 'влажные', 'влажн': 'влажные', 'сал': 'салфетка', 
          'дез': 'дезодарант', 'невид': 'невидимый', 'муж': 'мужской', 'з/щетка': 'зубная щетка', 'з/щ': 'зубная щетка',
          'мятн': 'мятный', 'прокл': 'прокладка', 'зуб': 'зубная', 'туалетнаябумага': 'туалентная бумага',
           'туалет': 'туалетная', 'эл': 'электронная', 'тб': 'туалетная бумага', 'матир': 'матирующий', 
          'тон': 'тональный', 'косме': 'косметичка', 'подводк': 'подводка', 'удл': 'удлинить', 'снят': 'снятия', 
           'быстрыйэф': 'быстрый эффект', 'кальц': 'кальций', 'укре': 'укрепления', 'укреплен': 'укрепления', 'сн': 'снятия',
          'компл': 'комплект', 'лит-ра': 'литература', 'кн': 'книга', 
           #'изд': 'издательство', 
           'сер': 'серия', 'уч': 'учебник', 
          'беременност': 'беременность', 'беременн': 'беременность', 'стер': 'стерильный', 'ant': 'антисептик', 
           'antisepti': 'антисептик', 'медиц': 'медицина', 'сте': 'стерильный', 'трубч': 'трубчатый', 'обув': 'обувь',
          'марл': 'марлевый', 'марлев': 'марлевый', 'мед': 'медицина', 'повяз': 'повязка', 'л/пл': 'лейкопластырь',
          'смотр': 'смотровые', 'стерил': 'стерильный', 'опр': 'опеделения', 'берем': 'беременность',
          'одн': 'одноразовый', 'однор': 'одноразовый', 'однораз': 'одноразовый', 'лейкопл': 'лейкопластырь', 
           'изг-е': 'изготовление', 'мрт': 'магниторезонансная терапия', 'кт': 'компьютерная томография', 
           'орт': 'ортопед', 'спец': 'специальный', 'хир': 'хирургия', 'одек': 'одеколон', 'туал': 'туалетная', 
           'п/в': 'парфюмерная вода', 'сыв': 'сыворотка', 'кож': 'кожа', 'вол': 'волосы', 'массажн': 'массаж', 
           'прочи': 'прочее',
            'шамп': 'шампунь', 'шп': 'шампунь', 'шампhs': 'шампунь', 'укреп': 'укрепление', 'золо': 'золотой',
          'бальз': 'бальзам', 'ш-нь': 'шампунь', 'жир': 'жирные', 'ослаб': 'ослабленных', 'секущ': 'секущиеся',
          'конд': 'кондиционер', 'кон': 'кондиционер', 'кондиц': 'кондиционер', 'бров': 'брови', 'сиян': 'сияние',
          'трав': 'травы', 'перх': 'перхоть', 'шмп': 'шампунь', 'жем': 'жемчужина', 'умыв': 'умывания', 
          'сыворот': 'сыворотка', 'брит': 'бритья', 'очищ': 'очищающий', 'чж': 'черная жемчужина', 'черн': 'черный', 
          'скр': 'скраб', 'касеты': 'кассеты', 'ноч': 'ночной', 'дн': 'дневной', 'восстан': 'восстанавливающая', 
          'интенс': 'интенсивный', 'натур': 'натуральный', 'барх': 'бархатные', 'жен': 'женская', 'освежающ': 'освежающая',
          'ж/мыло': 'женское мыло', 'д/рук': 'для рук', 'жид': 'жидкое', 'ванн': 'ванна', 'т/мыло': 'туалетное мыло',
          'пиявк': 'пиявка', 'шер': 'шерсть', 'трик': 'трикотажные', 'брюкик': 'брюки', 'брюкир': 'брюки', 
          'женс': 'женские', 'мальч': 'мальчик', 'мужска': 'мужская', 'джинс': 'джинсы', 'полуб-ки': 'полуботинки',
          'п/ботинки': 'полуботинки', 'рез': 'резина', 'сап': 'сапоги', 'ж': 'женская', 'д/об': 'для обуви', 
           'бюстгальт': 'бюстгальтер', 'бюст': 'бюстгальтер', 'бюстгалтер': 'бюстгальтер', 'колг': 'колготки',
          'кол': 'колготки', 'колготкиtgb': 'колготки', 'колготы': 'колготки', 'трус': 'трусы', 'трусики': 'трусы',
          'утепл': 'утепленный', 'футб': 'футболка', 'блуза': 'блузка', 'нос': 'носки',
          'пиджакк': 'пиджак', 'пиджакр': 'пиджак', 'укороч': 'укороченный', 'комфорт': 'комфортный', 'диз': 'дизельное', 
           'зим': 'зимняя', 'элит': 'элитный', 'легг': 'легинсы', 'шор': 'шорты', 'ветчинагрибы': 'ветчина грибы', 
           'замор': 'замороженная', 'кус': 'кусочек', 'ветчин': 'ветчина', 'пиццааппетитная': 'пицца', 
           'пиццаветчина': 'пицца ветчина', 'пиццаролл': 'пицца ролл', 'выпеченн': 'выпеченный', 'моцарел': 'моцарелла',
          'ветч': 'ветчина', 'pizza': 'пицца', 'pepperoni': 'пицца пепперони', 'capricciosa': 'пицца capricciosa', 
          'баварская': 'баварская пицца', 'пепперончини': 'пицца пепперони', 'пепперони': 'пепперони пицца', 
           'сыра': 'сыра пицца', 'телепицца': 'телепицца пицца', 'мексиканская': 'мексиканская пицца', 
           'капричеса': 'пицца капричеса', 'гавайская': 'гавайская пицца', 'филадельфия': 'филадельфия ролл', 
          'миниролл': 'мини ролл', 'копч': 'копченой', 'гамбургер': 'бургер', 'роял': 'роял бургер', 
           'роялбургер': 'роял бургер', 'сандвич': 'сэндвич', 'сендвич': 'сэндвич', 'фрешбургер': 'фреш бургер', 
           'шефбугер': 'шеф бургер', 'чикенсандвич': 'чикен сэндвич', 'шефбургер': 'шеф бургер', 'burger': 'бургер',
          'кур': 'куриные', 'остр': 'острый', 'жаренн': 'жаренный', 'лук': 'луковые', 'клас-кие': 'классические',
          'острмакнаггет': 'острые макнаггетс', 'свин': 'свинина', 'классик': 'классическая', 'тарел': 'тарелка',
          'чик': 'чикен', 'бек': 'бекон', 'макнаггетс': 'мак наггетс', 'нагетсы': 'наггетсы',
          'ланчбаскет': 'ланч баскет', 'ланчбокс': 'ланч бокс', 'штстрипсы': 'стрипсы', 'шткур': 'куриные', 
           'станд': 'стандартный', 'чикенбокс': 'чикен бокс', 'чикенбургер': 'чикен бургер', 
           'чикеннаггетсы': 'чикен наггетсы', 'чикенмак': 'чикен мак', 'чикенстар': 'чикен стар', 
           'чизбургер': 'чиз бургер сыр', 'макфреш': 'мак фреш', 'боксмастер': 'бокс мастер',
          'америк': 'американо', 'ассам': 'чай ассам', 'американоg': 'американо', 'анчан': 'чай анчан',
          'аффогато': 'кофе аффогато', 'capuccino': 'капучино', 'capuccino': 'капучино', 'бразил': 'бразильский',
          'гляссе': 'кофе гляссе', 'латте': 'кофе латте', 'латтэ': 'кофе латте',
           'капучино': 'кофе каппучино', 'американо': 'кофе американо',
          'каппучино': 'кофе каппучино', 'капучиноg': 'каппучино', 'коф': 'кофе', 'напит': 'напиток', 
           'кортадо': 'кофе кортадо', 'мокко': 'кофе мокко', 'майский': 'майский чай', 'эспрессо': 'кофе эспрессо', 
           'эспресо': 'кофе эспрессо', 'мол': 'молоко', 'раф': 'раф кофе', 'розлив': 'розливной', 'раст': 'растительный',
          'сенча': 'чай сенча','мокачино': 'кофе мокачино','моккачино': 'кофе мокачино', 'мокаччино': 'кофе мокачино',
          'ройбуш': 'чай ройбуш', 'фкапучино': 'капучино', 'капуччино': 'кофе каппучино', 'капучинно': 'кофе каппучино',
          'espresso': 'эспрессо', 'фраппе': 'кофе фраппе', 'рухуна': 'чай рухуна', 'амерекано': 'кофе американо',
          'биф': 'бифштекс', 'картофе': 'картофель', 
           'аджапсандали': 'блюдо из баклажан помидор сладкий перец лук чеснок', 
           'акцияшашлык': 'акция шашлык', 'балоньезе': 'паста макароны', 'биточки': 'биточек', 'блинный': 'блин', 
           'фрик': 'фрикаделька', 'бризоль': 'курица мясо свинина сыр', 'гаспачо': 'суп пюре помидор огурец', 
           'блинчик': 'блин',
           'доп': 'дополнительный', 'долма': 'виноград лист баранина масло свинина',
          'морск': 'морской', 'жар': 'жареный', 'котл': 'котлета', 'котлет': 'котлета', 'свеж': 'свежей',
          'фокачо': 'лепешка', 'фокача': 'лепешка', 'фокачча': 'лепешка', 'с-т': 'салат', 'с/т': 'салат', 
           'колб': 'колбаса', 'класс': 'классический', 'говяд': 'говядина', 'болоньезе': 'паста макароны', 
           'хрустящ': 'хрустящий', 'неж': 'нежный', 'дп': 'детское питание', 'черносл': 'чернослив', 
           'фрутоняня': 'фруто няня', 'земл': 'земляника', 'фрняня': 'фруто няня', 'детск': 'детский',
          'яблклуб': 'яблоко клубника', 'перс': 'персик', 'лесн': 'лесные', 'биойог': 'био йогурт', 'фрук': 'фруктовые',
          'фн': 'фруто няня', 'ф-н': 'фруто няня', 'спаг': 'спагетти макароны', 'молсмесь': 'молоко смесь',
          'баблукпюре': 'бабушкино лукошко пюре', 'баб': 'бабушкино', 'овсян': 'овсяное', 'черни': 'черничный',
          'молсмесьсимилакголд': 'молоко смесь симилак голд', 'неосвет': 'неосветленный', 'осветл': 'осветленный',
          'биотв': 'био творог', 'биотворог': 'био творог', 'бзмж': 'без заменителя молочного жира', 
           'бзмжмороженое': 'бзмж мороженое', 'блинч': 'блинчики', 'заморож': 'замороженный', 'бульмени': 'пельмени',
          'ваф': 'вафли', 'горяч': 'горячая', 'дес': 'десерт', 'краб': 'крабовые', 'змж': 'замороженный',
          'м/е': 'мороженое', 'мират': 'мираторг', 'м-ное': 'мороженое', 'морож': 'мороженое', 'мор': 'мороженое',
          'морожен': 'мороженое','мороженное': 'мороженое', 'карам': 'карамель', 'слоен': 'слоеное', 'стакан':'стаканчик',
          'биомороженое': 'био мороженое', 'капутс': 'капуста', 'филев': 'филе', 'пельм': 'пельмени', 'цыпл': 'цыпленок',
          'эск': 'эскимо', 'п/фаб': 'полуфабрикат', 'п/фабрикат': 'полуфабрикат', 'вар': 'вареная', 
          'в/к': 'великолужский', 'остан': 'останкино', 'сервел': 'сервелат', 'серв': 'сервелат',
          'сос': 'сосика', 'сосис': 'сосиска', 'сард': 'сарделька', 'к-са': 'колбаса', 'грудинкаособаявк': 'грудинка особая',
          'горох': 'горошек', 'горош': 'горошек', 'кваш': 'квашенная', 'квашеная': 'квашенная', 'конс': 'консервы',
          'кукуруз': 'кукуруза', 'кук': 'кукуруза', 'олив': 'оливка', 'верм': 'вермишель',
          'гречн': 'гречневая', 'каш': 'каша', 'овс': 'овсяная', 'кукруза': 'кукуруза', 'ячне': 'ячневая',
          'лапш': 'лапша', 'макарон': 'макароны', 'разрыхлит': 'разрыхлитель','спагет': 'спагетти', 
           'макпр': 'макароны', 'мак/изд': 'макароны изделия','пшеничн': 'пшеничная', 'рж': 'ржаная', 'гречка': 'греча',
          'гречневая': 'греча', 'рисовая': 'рис', 'быстр': 'быстрый', 'спагетти': 'макароны', 'фунчоза': 'макароны',
          'кетч': 'кетчуп', 'ванильн': 'ванильный', 'горчич': 'горчица',
          'подс': 'подсолнечное', 'подсол': 'подсолнечное', 'подсолн': 'подсолнечное', 'подсолнеч': 'подсолнечное',
          'подсолне': 'подсолнечное', 'слив': 'сливочное', 'пасх': 'пасхальный', 'сахарн': 'сахар',
          'наклейкнапасхальняйц': 'наклейка на пасхальные яйца', 'припр': 'приправа', 'повареная': 'поваренная',
          'при/ва': 'приправа', 'при-ва': 'приправа', 'прип': 'приправа','топинг': 'топпинг', 'золот': 'золотая',
          'злато': 'подсолнечное', 'молот': 'молотый', 'йогпит': 'йогурт питьевой', 'биойогурт': 'био йогурт',
          'биогурт': 'био йогурт', 'пит': 'питьевой', 'биокефир': 'био кефир', 'биоваренец': 'био варенец',
          'биоряж': 'био ряженка', 'биопродукт': 'био продукт', 'биоряженка': 'био ряженка','вкусн': 'вкусный',
          'термост': 'термостатный', 'фруатэ': 'фруктовый', 'фрутис': 'фруктовый', 'фруттис':'фруктовый',
          'натуральн': 'натуральный', 'клуб': 'клубника', 'кусоч': 'кусочек', 'греческ':'греческий', 
           'двухсл': 'двухслойный', 'клюкв': 'клюква', 'йогуртн': 'йогурт', 'йогуртнежн': 'йогурт нежный', 
          'йогуртнежныйперсикбзмж': 'йогурт нежный персик бзмж', 'йп': 'йогуртный продукт', 'кеф': 'кефир',
          'жирн': 'жирный', 'пюр': 'пюре', 'молоч': 'молочный', 'молочн': 'молочный', 'шок': 'шоколад',
          'м/з': 'майонез', 'м-з': 'майонез', 'майон': 'майонез', 'прованс': 'провансаль', 'прован':'провансаль',
          'детс': 'детское', 'отборн': 'отборное', 'отбор': 'отборное', 'пастер': 'пастеризованное', 
           'паст': 'пастеризованное', 'сгущ': 'сгущенка', 'сах': 'сахар', 'ультрапаст': 'ультра пастеризованное',
          'ультрап': 'ультра пастеризованное', 'ультр': 'ультра пастеризованное', 'ультроп': 'ультра пастеризованное',
          'ультрапаст': 'ультра пастеризованное', 'ультрпаст': 'ультра пастеризованное',
          'молокодомик': 'молоко домик', 'молокосод': 'молоко содержащий',  'молокосодерж': 'молоко содержащий',
          'сгущен': 'сгущенка', 'стерилиз': 'стерилизованное', 'кисломол': 'кисло молочный', 
           'сзмж': 'содержит заменитель молочных жиров', 'смет': 'сметана',
          'плав': 'плавленый', 'плавл': 'плавленый', 'плавлен': 'плавленый','российск': 'российский', 
          'россий': 'российский', 'глаз': 'глазированные', 'глазир': 'глазированные', 'глазиров': 'глазированные',
          'суфл': 'суфле', 'твор': 'творожный', 'творож': 'творожный', 'темн': 'темный', 'легк': 'легкий',
          'курин': 'куриное', 'цб': 'цыпленок бройлер', 'пф': 'полуфабрикат', 'шеи': 'шея', 'семеч': 'семечки',
           'сем': 'семечки', 'рыбк': 'рыбка', 'принглс': 'чипсы', 'лейс': 'чипсы', 'яблоч': 'яблочные', 
           'читос': 'чипсы', 'чипсоны': 'чипсы', 'хруст': 'хрустящий', 'pringles': 'чипсы', 'lays': 'чипсы', 
           'хрустим' : 'сухарики', 'короч': 'корочки', 'чип': 'чипсы', 'nachos': 'чипсы', 'beerka': 'арахис',
          'воронц': 'воронцовские', 'вял': 'вяленая', 'сопа': 'рыба сопа', 'сибас': 'рыба сибас',
          'вобла': 'рыба вобла', 'кета': 'рыба кета', 'камбала': 'рыба камбала', 'горбуша': 'рыба горбуша', 
           'сельдь':'рыба сельдь', 'анчоус': 'рыба анчоус', 'балык': 'рыба балык', 'вомер': 'рыба вомер',
          'дорада': 'рыба дорада', 'жерех': 'рыба жерех', 'голец': 'рыба голец', 'карп': 'рыба карп', 
           'килька': 'рыба килька', 'каракатица': 'рыба каракатица', 'карась': 'рыба карась', 
           'мореп': 'морепродукты', 'морепрод': 'морепродукты', 'морепродукт': 'морепродукты',
          'морепр': 'морепродукты', 'сайда': 'рыба сайда', 'мин/вода': 'минеральная вода',
           'мин-вода': 'минеральная вода', 'мин': 'минеральная', 'водаборжомистекло': 'вода боржоми стекло',
          'водапитьевая': 'вода питьевая', 'негаз': 'негазированная', 'негазиров': 'негазированная',
          'черног': 'черноголовка', 'минерал': 'минеральная', 'минер': 'минеральная', 'минеральн': 'минеральная',
          'питьев': 'питьевая', 'aqua': 'вода аква минерале', 'minerale': 'вода аква минерале', 'borjomi': 'вода боржоми',
          'анан': 'ананас', 'газир': 'газированная', 'свят': 'святой', 'г/в': 'газированная вода', 
           'газвода': 'газированная вода', 'энергетич': 'энергетический', 'энергет': 'энергетический', 'энер': 'энергетический', 
          'энег': 'энергетик', 'эн': 'энергетический', 'н/к': 'напиток', 'н-к': 'напиток', 'швеп': 'швепс лимонад', 
           'fuse': 'фьюз', 'tea': 'чай', 'минвода': 'минеральная вода', 'сокосод': 'сокосодержащий',
          'сокосодер': 'сокосодержащий', 'сильногаз': 'сильногазированный', 'сокосодерж': 'сокосодержащий',
          'шокол': 'шоколад', 'бат': 'батончик', 'ш-д': 'шоколад', 'ш/д': 'шоколад', 'ш': 'шоколад', 'холс': 'холлс',
          'хлебобулочн': 'хлебобулочное', 'хлебобул': 'хлебобулочное', 'хлебо-булочное': 'хлебобулочное', 
           'хлебо/булочное': 'хлебобулочное', 'ржано': 'ржаной', 'пшенич': 'пшеничный', 'пш': 'пшеничный', 'пшен': 'пшеничный',
          'бул': 'булка', 'х/б': 'хлебобулочное', 'изд': 'изделие', 'пир': 'пирог', 'печен': 'печенье', 'печ': 'печенье',
          'круас': 'круассан', 'круасан': 'круассан', 'круасс': 'круассан', 'круасаны': 'круассан', 'конф': 'конфеты',
          'зефирн': 'зефир', 'резина': 'резинка', 'марм': 'мармелад', 'свадебные': 'свадьба', 
           'танцпола': 'танцпол', 'диско': 'дискотека', 'празднич': 'праздник', 'пневмохлопушка': 'пневмо хлопушка', 
          'пневмохл': 'пневмо хлопушка', 'поздравл': 'поздравление', 'праздничные': 'праздник', 
           'праздничный': 'праздник', 
           'русскаябаня': 'русская баня', 'туристический': 'турист', 'вых': 'выходной', 'н-р': 'набор',
          'хвс': 'холодное водоснабжение', 'вдго': 'внутридомовое газовое оборудование',
          'вкго': 'внутриквартирное газовое оборудование', 'капремонт': 'капитальный ремонт',
          'жбо': 'жидкие бытовые отходы', 'гвс': 'горячее водоснабжение', 
           'сои': 'содержание общедомового имущества', 'ор': 'общедомовые ресурсы', 
           'тко': 'твердые бытовые отходы', 'моп': 'места общего пользования', 
           'хв': 'холодное водоснабжение', 'мкд': 'многоквартиный дом', 'ду': 'управление дома',
          'одн': 'общедомовые нужды', 'одпу': 'общедомовые приборы учета', 'гп': 'государственное предприятие',
          'оди': 'общедомовые нужды', 'вгдо': 'внутридомовое газовое оборудование', 
           'жкх': 'жилищно коммунальное хозяйство', 'ои': 'общее имущество',
          'ипу': 'индивидуальные приборы учета', 'квт': 'киловатт час',
          'жку': 'жилищно коммунальные услуги', 'использ': 'использование', 'содерж': 'содержать', 'общ': 'общее',
          'муп': 'муниципальное унитарное предприятие', 'НДС': 'налог на добавленную стоимость',
          'соджилья': 'содержание жилья', 'пк': 'повыщающий коэффициент', 'дек': 'декабрь', 'окт': 'октябрь',
          'апр': 'апрель', 'повыш': 'повышающий', 'нояб': 'ноябрь', 'отвед': 'отведение', 'обсл': 'обслуживание',
          'квартплат': 'квартирная плата', 'элеснаб': 'электроснабжение', 'взросл': 'взрослый', 
           'ласты': 'бассейн ласты', 'протеиновый': 'протеиновый энергетический спортивное питание',
          'chiкabar': 'энергетический протеиновый спортивное питание набор массы', 
          'monohydrate': 'смесь для ускорения роста мышц протеин спортивное питание заменитель набор массы',
          'energon': 'энергетическое спортивное питание заменитель', 'чамп': 'энергетическое спортивное питание', 
          'белок': 'спортивное питание', 'фитнес': 'фитнес спортивный', 'smart': 'спортивное питание', 
           'eemb': 'энергетический протеиновый спортивное питание для спорта набор массы', 
           'eemm': 'энергетический протеиновый спортивное питание для спорта набор массы', 
           'nutrition': 'протетиновое спортивное питание', 'quest': 'энергетический протеивновый набор массы' , 
           'prot': 'спортивное питание заменитель', 'спортик': 'спортивное питание набор массы', 'турбослим': 'спортивное питание заменитель',
          'соевый': 'спортивное питание заменитель', 'пребиосвит': 'спортивное питание заменитель',
          'samyun': 'набор массы спортивное питание энергетическое', 'wan': 'спортивное питание', 
           'muscle': 'набор массы мускулы спортивное питание',
          'alpinistiк': 'спортивный тренировочный инвентарь', 
           'трос': 'спортивный тренировочный инвентарь силовые тренировки',
           'резинов': 'резиновая', 'скакалка': 'спортивный инвентарь для тренировок прыжки спорт',
          'weight': 'спортивный тренировочный инвентарь силовые тренировки', 
           'weights': 'спортивный тренировочный инвентарь силовые тренировки',
           'гантель': 'спортивный тренировочный инвентарь силовые тренировки', 
           'гантели': 'спортивный тренировочный инвентарь силовые тренировки',
          'гимнастический': 'спортивный тренировочный инвентарь гимнастические тренировки', 
           'гимнастич': 'гимнастический', 'эспандер': 'спортивный тренировочный инвентарь гимнастические тренировки',
          'training': 'спортивный тренировочный инвентарь силовые тренировки', 
          'обруч': 'спортивный тренировочный инвентарь гимнастические тренировки', 
          'тяга': 'спортивный тренировочный инвентарь силовые тренировки', 'рулевая': 'силовые тренировки',
          'утяжелители': 'спортивный тренировочный инвентарь силовые тренировки', 
           'пластмасс': 'пластмассовый', 'silapro': 'спортивный тренировочный инвентарь',
          'starfit': 'спортивный тренировочный инвентарь силовые тренировки', 'оберт':'обертка',
          'пак': 'пакет', 'подар': 'подарок', 'творч': 'творчество', 
           'раскраска': 'художественные принадлежности для творчества  и арт искусство рисование',
         
          'мелки': 'художественные принадлежности для творчества  и арт искусство рисование', 
          'скетчбук': 'художественные принадлежности для творчества  и арт искусство рисование',
          'фломастеры': 'художественные принадлежности для творчества  и арт искусство рисование',
          'фломастер': 'художественные принадлежности для творчества  и арт искусство рисование',
          'трафарет': 'художественные принадлежности для творчества  и арт искусство рисование',
         'разбавитель': 'художественные принадлежности для творчества  и арт искусство рисование', 
           'холст': 'художественные принадлежности для творчества  и арт искусство рисование',
          'раскр': 'раскраска', 'творчество': 'художественные принадлежности для творчества  и арт искусство рисование',
          'альбом': 'художественные принадлежности для творчества  и арт искусство рисование',
          'акварель': 'художественные принадлежности для творчества  и арт искусство рисование',
          'акрил': 'художественные принадлежности для творчества  и арт искусство рисование',
          'мольберт': 'художественные принадлежности для творчества  и арт искусство рисование',
          'акварель': 'художественные принадлежности для творчества  и арт искусство рисование',
          
          'акварель': 'художественные принадлежности для творчества  и арт искусство рисование',
          'кистей': 'художественные принадлежности для творчества  и арт искусство рисование', 
          'держат': 'держатель', 'магнит': 'магнитный', 'реш': 'решетка',
          'гофра': 'трубчатые строительные материалы сантехника', 
           'аэратор': 'трубчатые строительные материалы сантехника', 
           'втулка': 'трубчатые строительные материалы сантехника', 
          'сифон': 'трубчатые строительные материалы сантехника', 
           'гофрированный': 'трубчатые строительные материалы сантехника',
          'смеситель': 'трубчатые строительные материалы сантехника', 
          'клапан': 'трубчатые строительные материалы сантехника', 
           'гофротрубка': 'трубчатые строительные материалы сантехника',
          'кран': 'трубчатые строительные материалы сантехника', 'сантехника': 'трубчатые строительные материалы сантехника',
          'шланг': 'трубчатые строительные материалы сантехника',
          'унитаза': 'трубчатые строительные материалы сантехника', 
           'стоки':'трубчатые строительные материалы сантехника', 'cифон': 'трубчатые строительные материалы сантехника',
          'сантех':'трубчатые строительные материалы сантехника', 'излив': 'трубчатые строительные материалы сантехника',
          'арматура': 'трубчатые строительные материалы сантехника','умывальник':'трубчатые строительные материалы сантехника',
          'лейка': 'трубчатые строительные материалы сантехника','дюб': 'дюбель'}
# шамп: шампунь шампанское

Так мы получаем предобработанный текст.

In [42]:
dct_, tr_, = prepare(train, perevod, l, maslin, prazdnik, do_lemma=False)

STEP 1
STEP 2
STEP 3
STEP 4
STEP 5
STEP 6
MAPPING....


## Word2Vec

Так мы обучаем `Word2Vec`

In [ ]:
from gensim.models import Word2Vec
import pickle

In [ ]:
vocab_tokemizer = {k: w.split(' ') for k, w in enumerate(tr_['preprocess'])}

In [ ]:
model = Word2Vec(list(vocab_tokemizer.values()), # data for model to train on
                 size=300,         # embedding vector size
                 min_count=7,     # consider words that occured at least 5 times
                 window=7).wv     # define context as a 3-word window around the target word

In [ ]:
pickle.dump(model, open('word2vec', 'wb'))

Откроем обученный нами `word2vec`

In [17]:
m = pickle.load(open('word2vec', 'rb'))

Получаем эмбеддинги предложения.

In [18]:
def question_to_vec(question, embeddings, dim=300, first=True):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    words = question.split(' ') #your code
    # убрать знак вопроса, если он есть
    n_known = 0
    result = np.array([0] * dim, dtype=float)
    
    if first:
        word = words[0]
        if word in embeddings:
            result += embeddings[word] #your code
            n_known += 1
    else:
        for word in words:
            if word in embeddings:
                result += embeddings[word] #your code
                n_known += 1
            
    if n_known != 0:
        return result / n_known #your code
    else:
        return result

In [46]:
unique_items = train_data.drop_duplicates('item_name')
dct2_, tr2_, = prepare(unique_items, perevod, l, maslin, prazdnik, do_lemma=False)

STEP 1
STEP 2
STEP 3
STEP 4
STEP 5
STEP 6
MAPPING....


Примеры предобработки текста:

In [38]:
unique_items.loc[72]['item_name'], unique_items.loc[72]['preprocess']

('Спинки варено-копченые из мяса ЦБ в/у охладенные',
 'спинки варено копченые из мяса цыпленок бройлер вакуумная упаковка охладенные')

In [41]:
unique_items.loc[221]['item_name'], unique_items.loc[221]['preprocess']

('Брюки трик. женские', 'брюки трикотажные женские')

Полуичим эмбеддинги.

In [20]:
%%time

X = np.array([question_to_vec(question=question, embeddings=m, dim=300, first=False) for question in tr2_['preprocess']])
y = unique_items['category_id'].values

Wall time: 3.4 s


Составим таблицу эмбеддингов.

In [11]:
dat = pd.DataFrame(np.concatenate((X, 
                                   tr2_[['receipt_dayofweek', 'item_quantity', 'item_price', 'item_nds_rate']].values, 
                                   y[:, np.newaxis]), axis=1))
columns = [f'feature_{x}' for x in np.arange(0, 300)] + \
['receipt_dayofweek', 'item_quantity', 'item_price', 'item_nds_rate'] + \
['category_id']
dat.columns = columns

In [12]:
dat.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_295,feature_296,feature_297,feature_298,feature_299,receipt_dayofweek,item_quantity,item_price,item_nds_rate,category_id
0,0.054561,-0.392966,0.410688,0.116836,0.989019,1.157157,1.632287,-0.352680,-0.509084,0.689676,...,1.274330,-1.676105,-0.193540,1.174354,0.899041,6.0,2.0,8.0,2.0,78.0
1,0.376635,-1.049812,1.021654,-0.610256,-0.561202,-0.090528,1.225579,0.942012,-0.220868,1.147172,...,-1.592884,-1.578708,0.510537,-0.795046,0.296348,4.0,1.0,4.0,1.0,71.0
2,0.020388,-1.286427,0.521235,-1.633261,0.155876,-0.660067,0.093485,0.784330,0.392229,-0.063603,...,-1.068598,-1.367852,-0.088262,-1.153419,0.858392,4.0,1.0,4.0,1.0,71.0
3,-0.993593,0.944048,-0.093030,-0.828796,-0.848058,0.692314,0.216841,-1.680906,-1.517059,-0.495888,...,-0.079874,0.545516,0.435811,-0.417459,0.429165,5.0,1.0,12.0,1.0,70.0
4,1.044952,0.706317,0.509403,0.475204,1.119540,1.249804,1.099545,-0.517986,-0.934430,0.830007,...,1.256626,-0.397411,0.466691,0.839089,0.332216,3.0,1.0,7.0,-1.0,84.0


На этом можно обучать модели, например `KNN`.

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import f1_score

In [26]:
knn = KNeighborsClassifier(n_neighbors=9)
folds = KFold(8, 
              shuffle=True, 
              random_state=0)

In [27]:
predicts = cross_val_predict(knn, 
                             X, 
                             y, 
                             cv=folds, 
                             n_jobs=8, 
                             method='predict')

In [29]:
score = f1_score(y, predicts, average='weighted')
print(score)

0.7549749233089824


Посмотрим вообще на эмбеддинги.

In [13]:
m.most_similar('кефир')

[('ряженка', 0.718873143196106),
 ('варенец', 0.58260178565979),
 ('снежок', 0.4844396710395813),
 ('ультрапастеризованное', 0.46592074632644653),
 ('биоматрикс', 0.4627859890460968),
 ('простокваша', 0.43150943517684937),
 ('бифидок', 0.40915602445602417),
 ('питьевое', 0.39816442131996155),
 ('кефирный', 0.3964994251728058),
 ('пастеризованное', 0.38972562551498413)]

In [27]:
m.most_similar('гайка')

[('болт', 0.7063924074172974),
 ('шпилька', 0.5963376760482788),
 ('гайкой', 0.5931398868560791),
 ('шплинт', 0.5835789442062378),
 ('резьбовая', 0.581788957118988),
 ('оцинкованная', 0.5693467855453491),
 ('din', 0.561095118522644),
 ('контргайка', 0.5568018555641174),
 ('гровер', 0.5169054269790649),
 ('нейлоновой', 0.5123203992843628)]

## SVMClassifier + TFIDF

Так мы делаем предсказание с помощью модели из пункта `III.3`

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
unique_items = train_data.drop_duplicates('item_name')
d, x =  prepare(unique_items, perevod, l, maslin, prazdnik, do_lemma=False)

tfidf = TfidfVectorizer(
    max_features=800000, 
    ngram_range=(1, 6), 
    stop_words ='russian',
    analyzer="char_wb", 
    norm = 'l2'
)

x['new'] = x['preprocess'] + ' ' + x['item_name']
X_train = tfidf.fit_transform(x.new)
y_train = x['category_id']
clf = LinearSVC(cl)
folds = KFold(8, 
              shuffle=True, 
              random_state=0)

STEP 1
STEP 2
STEP 3
STEP 4
STEP 5
STEP 6
MAPPING....


c:\users\shiro\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\shiro\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [10]:
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [15]:
predicts = cross_val_predict(clf, 
                             X_train, 
                             y_train, 
                             cv=folds, 
                             n_jobs=8, 
                             method='predict')

In [11]:
pickle.dump(clf, open('new_sol_2/clf_task1', 'wb'))
pickle.dump(tfidf, open('new_sol_2/tfidf', 'wb'))

In [16]:
from sklearn.metrics import f1_score

score = f1_score(y_train, predicts, average='weighted')
print(score)

0.845884989392697


In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_train, predicts))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      2356
           1       0.93      0.89      0.91        28
           2       0.97      0.89      0.92       316
           3       0.88      0.87      0.88       109
           4       0.70      0.64      0.67       225
           6       0.55      0.38      0.44        64
           7       0.94      0.98      0.96       224
           9       0.90      0.98      0.94       104
          11       0.60      0.53      0.56        47
          12       0.78      0.69      0.73       182
          13       0.71      0.59      0.65        37
          19       0.74      0.50      0.60        74
          20       0.98      0.88      0.93        59
          24       0.65      0.47      0.55        68
          26       0.20      0.09      0.13        22
          27       0.72      0.68      0.70        38
          29       0.78      0.89      0.83       129
          30       0.78    

Видим, что есть отличная разделимость примерно половины классов, но на ухудшают результат очень плохо разделимые классые - например класс `26` - органайзеры, ножницы, бумага - очень перекрывающиеся c другими классами. В этом и проблема представления `tfidf` - контекст и порядок слов не учитываются. Поэтому, нужно применять другие модели. А какие - узнаем у победителей.

Спасибо!